<project title="FastHTML" summary='FastHTML is a python library which brings together Starlette, Uvicorn, HTMX, and fastcore&#39;s `FT` "FastTags" into a library for creating server-rendered hypermedia applications. The `FastHTML` class itself inherits from `Starlette`, and adds decorator-based routing with many additions, Beforeware, automatic `FT` to HTML rendering, and much more.'>Things to remember when writing FastHTML apps:

- Although parts of its API are inspired by FastAPI, it is *not* compatible with FastAPI syntax and is not targeted at creating API services
- FastHTML includes support for Pico CSS and the fastlite sqlite library, although using both are optional; sqlalchemy can be used directly or via the fastsql library, and any CSS framework can be used. Support for the Surreal and css-scope-inline libraries are also included, but both are optional
- FastHTML is compatible with JS-native web components and any vanilla JS library, but not with React, Vue, or Svelte
- Use `serve()` for running uvicorn (`if __name__ == "__main__"` is not needed since it's automatic)
- When a title is needed with a response, use `Titled`; note that that already wraps children in `Container`, and already includes both the meta title as well as the H1 element.<docs><doc title="FastHTML concise guide" desc="A brief overview of idiomatic FastHTML apps"># Concise reference



## About FastHTML

``` python
from fasthtml.common import *
```

FastHTML is a python library which brings together Starlette, Uvicorn,
HTMX, and fastcore’s `FT` “FastTags” into a library for creating
server-rendered hypermedia applications. The
[`FastHTML`](https://www.fastht.ml/docs/api/core.html#fasthtml) class
itself inherits from `Starlette`, and adds decorator-based routing with
many additions, Beforeware, automatic `FT` to HTML rendering, and much
more.

Things to remember when writing FastHTML apps:

- *Not* compatible with FastAPI syntax; FastHTML is for HTML-first apps,
  not API services (although it can implement APIs too)
- FastHTML includes support for Pico CSS and the fastlite sqlite
  library, although using both are optional; sqlalchemy can be used
  directly or via the fastsql library, and any CSS framework can be
  used. MonsterUI is a richer FastHTML-first component framework with
  similar capabilities to shadcn
- FastHTML is compatible with JS-native web components and any vanilla
  JS library, but not with React, Vue, or Svelte
- Use [`serve()`](https://www.fastht.ml/docs/api/core.html#serve) for
  running uvicorn (`if __name__ == "__main__"` is not needed since it’s
  automatic)
- When a title is needed with a response, use
  [`Titled`](https://www.fastht.ml/docs/api/xtend.html#titled); note
  that that already wraps children in
  [`Container`](https://www.fastht.ml/docs/api/pico.html#container), and
  already includes both the meta title as well as the H1 element.

## Minimal App

The code examples here use fast.ai style: prefer ternary op, 1-line
docstring, minimize vertical space, etc. (Normally fast.ai style uses
few if any comments, but they’re added here as documentation.)

A minimal FastHTML app looks something like this:

``` python
# Meta-package with all key symbols from FastHTML and Starlette. Import it like this at the start of every FastHTML app.
from fasthtml.common import *
# The FastHTML app object and shortcut to `app.route`
app,rt = fast_app()

# Enums constrain the values accepted for a route parameter
name = str_enum('names', 'Alice', 'Bev', 'Charlie')

# Passing a path to `rt` is optional. If not passed (recommended), the function name is the route ('/foo')
# Both GET and POST HTTP methods are handled by default
# Type-annotated params are passed as query params (recommended) unless a path param is defined (which it isn't here)
@rt
def foo(nm: name):
    # `Title` and `P` here are FastTags: direct m-expression mappings of HTML tags to Python functions with positional and named parameters. All standard HTML tags are included in the common wildcard import.
    # When a tuple is returned, this returns concatenated HTML partials. HTMX by default will use a title HTML partial to set the current page name. HEAD tags (e.g. Meta, Link, etc) in the returned tuple are automatically placed in HEAD; everything else is placed in BODY.
    # FastHTML will automatically return a complete HTML document with appropriate headers if a normal HTTP request is received. For an HTMX request, however, just the partials are returned.
    return Title("FastHTML"), H1("My web app"), P(f"Hello, {name}!")
# By default `serve` runs uvicorn on port 5001. Never write `if __name__ == "__main__"` since `serve` checks it internally.
serve()
```

To run this web app:

``` bash
python main.py  # access via localhost:5001
```

## FastTags (aka FT Components or FTs)

FTs are m-expressions plus simple sugar. Positional params map to
children. Named parameters map to attributes. Aliases must be used for
Python reserved words.

``` python
tags = Title("FastHTML"), H1("My web app"), P(f"Let's do this!", cls="myclass")
tags
```

    (title(('FastHTML',),{}),
     h1(('My web app',),{}),
     p(("Let's do this!",),{'class': 'myclass'}))

This example shows key aspects of how FTs handle attributes:

``` python
Label(
    "Choose an option", 
    Select(
        Option("one", value="1", selected=True),  # True renders just the attribute name
        Option("two", value=2, selected=False),   # Non-string values are converted to strings. False omits the attribute entirely
        cls="selector", id="counter",             # 'cls' becomes 'class'
        **{'@click':"alert('Clicked');"},         # Dict unpacking for attributes with special chars
    ),
    _for="counter",                               # '_for' becomes 'for' (can also use 'fr')
)
```

Classes with `__ft__` defined are rendered using that method.

``` python
class FtTest:
    def __ft__(self): return P('test')
    
to_xml(FtTest())
```

    '<p>test</p>\n'

You can create new FTs by importing the new component from
`fasthtml.components`. If the FT doesn’t exist within that module,
FastHTML will create it.

``` python
from fasthtml.components import Some_never_before_used_tag

Some_never_before_used_tag()
```

``` html
<some-never-before-used-tag></some-never-before-used-tag>
```

FTs can be combined by defining them as a function.

``` python
def Hero(title, statement): return Div(H1(title),P(statement), cls="hero")
to_xml(Hero("Hello World", "This is a hero statement"))
```

    '<div class="hero">\n  <h1>Hello World</h1>\n  <p>This is a hero statement</p>\n</div>\n'

When handling a response, FastHTML will automatically render FTs using
the `to_xml` function.

``` python
to_xml(tags)
```

    '<title>FastHTML</title>\n<h1>My web app</h1>\n<p class="myclass">Let&#x27;s do this!</p>\n'

## JS

The [`Script`](https://www.fastht.ml/docs/api/xtend.html#script)
function allows you to include JavaScript. You can use Python to
generate parts of your JS or JSON like this:

``` python
# In future snippets this import will not be shown, but is required
from fasthtml.common import * 
app,rt = fast_app(hdrs=[Script(src="https://cdn.plot.ly/plotly-2.32.0.min.js")])
# `index` is a special function name which maps to the `/` route. 
@rt
def index():
    data = {'somedata':'fill me in…'}
    # `Titled` returns a title tag and an h1 tag with the 1st param, with remaining params as children in a `Main` parent.
    return Titled("Chart Demo", Div(id="myDiv"), Script(f"var data = {data}; Plotly.newPlot('myDiv', data);"))
# In future snippets `serve() will not be shown, but is required
serve()
```

Prefer Python whenever possible over JS. Never use React or shadcn.

## fast_app hdrs

``` python
# In future snippets we'll skip showing the `fast_app` call if it has no params
app, rt = fast_app(
    pico=False, # The Pico CSS framework is included by default, so pass `False` to disable it if needed. No other CSS frameworks are included.
    # These are added to the `head` part of the page for non-HTMX requests.
    hdrs=(
        Link(rel='stylesheet', href='assets/normalize.min.css', type='text/css'),
        Link(rel='stylesheet', href='assets/sakura.css', type='text/css'),
        Style("p {color: red;}"),
        # `MarkdownJS` and `HighlightJS` are available via concise functions
        MarkdownJS(), HighlightJS(langs=['python', 'javascript', 'html', 'css']),
        # by default, all standard static extensions are served statically from the web app dir,
        #   which can be modified using e.g `static_path='public'`
        )
)

@rt
def index(req): return Titled("Markdown rendering example",
                              # This will be client-side rendered to HTML with highlight-js
                              Div("*hi* there",cls="marked"),
                              # This will be syntax highlighted
                              Pre(Code("def foo(): pass")))
```

## Responses

Routes can return various types:

1.  FastTags or tuples of FastTags (automatically rendered to HTML)
2.  Standard Starlette responses (used directly)
3.  JSON-serializable types (returned as JSON in a plain text response)

``` python
@rt("/{fname:path}.{ext:static}")
async def serve_static_file(fname:str, ext:str): return FileResponse(f'public/{fname}.{ext}')

app, rt = fast_app(hdrs=(MarkdownJS(), HighlightJS(langs=['python', 'javascript'])))
@rt
def index(): 
    return Titled("Example",
                  Div("*markdown* here", cls="marked"),
                  Pre(Code("def foo(): pass")))
```

Route functions can be used in attributes like `href` or `action` and
will be converted to paths. Use `.to()` to generate paths with query
parameters.

``` python
@rt
def profile(email:str): return fill_form(profile_form, profiles[email])

profile_form = Form(action=profile)(
    Label("Email", Input(name="email")),
    Button("Save", type="submit")
)

user_profile_path = profile.to(email="user@example.com")  # '/profile?email=user%40example.com'
```

``` python
from dataclasses import dataclass

app,rt = fast_app()
```

When a route handler function is used as a fasttag attribute (such as
`href`, `hx_get`, or `action`) it is converted to that route’s path.
[`fill_form`](https://www.fastht.ml/docs/api/components.html#fill_form)
is used to copy an object’s matching attrs into matching-name form
fields.

``` python
@dataclass
class Profile: email:str; phone:str; age:int
email = 'john@example.com'
profiles = {email: Profile(email=email, phone='123456789', age=5)}
@rt
def profile(email:str): return fill_form(profile_form, profiles[email])

profile_form = Form(method="post", action=profile)(
        Fieldset(
            Label('Email', Input(name="email")),
            Label("Phone", Input(name="phone")),
            Label("Age", Input(name="age"))),
        Button("Save", type="submit"))
```

## Testing

We can use `TestClient` for testing.

``` python
from starlette.testclient import TestClient
```

``` python
path = "/profile?email=john@example.com"
client = TestClient(app)
htmx_req = {'HX-Request':'1'}
print(client.get(path, headers=htmx_req).text)
```

    <form enctype="multipart/form-data" method="post" action="/profile"><fieldset><label>Email       <input name="email" value="john@example.com">
    </label><label>Phone       <input name="phone" value="123456789">
    </label><label>Age       <input name="age" value="5">
    </label></fieldset><button type="submit">Save</button></form>

## Form Handling and Data Binding

When a dataclass, namedtuple, etc. is used as a type annotation, the
form body will be unpacked into matching attribute names automatically.

``` python
@rt
def edit_profile(profile: Profile):
    profiles[email]=profile
    return RedirectResponse(url=path)

new_data = dict(email='john@example.com', phone='7654321', age=25)
print(client.post("/edit_profile", data=new_data, headers=htmx_req).text)
```

    <form enctype="multipart/form-data" method="post" action="/profile"><fieldset><label>Email       <input name="email" value="john@example.com">
    </label><label>Phone       <input name="phone" value="7654321">
    </label><label>Age       <input name="age" value="25">
    </label></fieldset><button type="submit">Save</button></form>

## fasttag Rendering Rules

The general rules for rendering children inside tuples or fasttag
children are: - `__ft__` method will be called (for default components
like `P`, `H2`, etc. or if you define your own components) - If you pass
a string, it will be escaped - On other python objects, `str()` will be
called

If you want to include plain HTML tags directly into e.g. a `Div()` they
will get escaped by default (as a security measure to avoid code
injections). This can be avoided by using `Safe(...)`, e.g to show a
data frame use `Div(NotStr(df.to_html()))`.

## Exceptions

FastHTML allows customization of exception handlers.

``` python
def not_found(req, exc): return Titled("404: I don't exist!")
exception_handlers = {404: not_found}
app, rt = fast_app(exception_handlers=exception_handlers)
```

## Cookies

We can set cookies using the
[`cookie()`](https://www.fastht.ml/docs/api/core.html#cookie) function.

``` python
@rt
def setcook(): return P(f'Set'), cookie('mycookie', 'foobar')
print(client.get('/setcook', headers=htmx_req).text)
```

     <p>Set</p>

``` python
@rt
def getcook(mycookie:str): return f'Got {mycookie}'
# If handlers return text instead of FTs, then a plaintext response is automatically created
print(client.get('/getcook').text)
```

    Got foobar

FastHTML provide access to Starlette’s request object automatically
using special `request` parameter name (or any prefix of that name).

``` python
@rt
def headers(req): return req.headers['host']
```

## Request and Session Objects

FastHTML provides access to Starlette’s session middleware automatically
using the special `session` parameter name (or any prefix of that name).

``` python
@rt
def profile(req, sess, user_id: int=None):
    ip = req.client.host
    sess['last_visit'] = datetime.now().isoformat()
    visits = sess.setdefault('visit_count', 0) + 1
    sess['visit_count'] = visits
    user = get_user(user_id or sess.get('user_id'))
    return Titled(f"Profile: {user.name}", 
                  P(f"Visits: {visits}"), 
                  P(f"IP: {ip}"),
                  Button("Logout", hx_post=logout))
```

Handler functions can return the
[`HtmxResponseHeaders`](https://www.fastht.ml/docs/api/core.html#htmxresponseheaders)
object to set HTMX-specific response headers.

``` python
@rt
def htmlredirect(app): return HtmxResponseHeaders(location="http://example.org")
```

## APIRouter

[`APIRouter`](https://www.fastht.ml/docs/api/core.html#apirouter) lets
you organize routes across multiple files in a FastHTML app.

``` python
# products.py
ar = APIRouter()

@ar
def details(pid: int): return f"Here are the product details for ID: {pid}"

@ar
def all_products(req):
    return Div(
        Div(
            Button("Details",hx_get=details.to(pid=42),hx_target="#products_list",hx_swap="outerHTML",),
        ), id="products_list")
```

``` python
# main.py
from products import ar,all_products

app, rt = fast_app()
ar.to_app(app)

@rt
def index():
    return Div(
        "Products",
        hx_get=all_products, hx_swap="outerHTML")
```

## Toasts

Toasts can be of four types:

- info
- success
- warning
- error

Toasts require the use of the `setup_toasts()` function, plus every
handler needs:

- The session argument
- Must return FT components

``` python
setup_toasts(app)

@rt
def toasting(session):
    add_toast(session, f"cooked", "info")
    add_toast(session, f"ready", "success")
    return Titled("toaster")
```

`setup_toasts(duration)` allows you to specify how long a toast will be
visible before disappearing.10 seconds.

Authentication and authorization are handled with Beforeware, which
functions that run before the route handler is called.

## Auth

``` python
def user_auth_before(req, sess):
    # `auth` key in the request scope is automatically provided to any handler which requests it and can not be injected
    auth = req.scope['auth'] = sess.get('auth', None)
    if not auth: return RedirectResponse('/login', status_code=303)

beforeware = Beforeware(
    user_auth_before,
    skip=[r'/favicon\.ico', r'/static/.*', r'.*\.css', r'.*\.js', '/login', '/']
)

app, rt = fast_app(before=beforeware)
```

## Server-Side Events (SSE)

FastHTML supports the HTMX SSE extension.

``` python
import random
hdrs=(Script(src="https://unpkg.com/htmx-ext-sse@2.2.3/sse.js"),)
app,rt = fast_app(hdrs=hdrs)

@rt
def index(): return Div(hx_ext="sse", sse_connect="/numstream", hx_swap="beforeend show:bottom", sse_swap="message")

# `signal_shutdown()` gets an event that is set on shutdown
shutdown_event = signal_shutdown()

async def number_generator():
    while not shutdown_event.is_set():
        data = Article(random.randint(1, 100))
        yield sse_message(data)

@rt
async def numstream(): return EventStream(number_generator())
```

## Websockets

FastHTML provides useful tools for HTMX’s websockets extension.

``` python
# These HTMX extensions are available through `exts`:
#   head-support preload class-tools loading-states multi-swap path-deps remove-me ws chunked-transfer
app, rt = fast_app(exts='ws')

def mk_inp(): return Input(id='msg', autofocus=True)

@rt
async def index(request):
    # `ws_send` tells HTMX to send a message to the nearest websocket based on the trigger for the form element
    cts = Div(
        Div(id='notifications'),
        Form(mk_inp(), id='form', ws_send=True),
        hx_ext='ws', ws_connect='/ws')
    return Titled('Websocket Test', cts)

async def on_connect(send): await send(Div('Hello, you have connected', id="notifications"))
async def on_disconnect(ws): print('Disconnected!')

@app.ws('/ws', conn=on_connect, disconn=on_disconnect)
async def ws(msg:str, send):
    # websocket hander returns/sends are treated as OOB swaps
    await send(Div('Hello ' + msg, id="notifications"))
    return Div('Goodbye ' + msg, id="notifications"), mk_inp()
```

Sample chatbot that uses FastHTML’s `setup_ws` function:

``` py
app = FastHTML(exts='ws')
rt = app.route
msgs = []

@rt('/')
def home():
    return Div(hx_ext='ws', ws_connect='/ws')(
        Div(Ul(*[Li(m) for m in msgs], id='msg-list')),
        Form(Input(id='msg'), id='form', ws_send=True)
    )

async def ws(msg:str):
    msgs.append(msg)
    await send(Ul(*[Li(m) for m in msgs], id='msg-list'))

send = setup_ws(app, ws)
```

### Single File Uploads

[`Form`](https://www.fastht.ml/docs/api/xtend.html#form) defaults to
“multipart/form-data”. A Starlette UploadFile is passed to the handler.

``` python
upload_dir = Path("filez")

@rt
def index():
    return (
        Form(hx_post=upload, hx_target="#result")(
            Input(type="file", name="file"),
            Button("Upload", type="submit")),
        Div(id="result")
    )

# Use `async` handlers where IO is used to avoid blocking other clients
@rt
async def upload(file: UploadFile):
    filebuffer = await file.read()
    (upload_dir / file.filename).write_bytes(filebuffer)
    return P('Size: ', file.size)
```

For multi-file, use `Input(..., multiple=True)`, and a type annotation
of `list[UploadFile]` in the handler.

## Fastlite

Fastlite and the MiniDataAPI specification it’s built on are a
CRUD-oriented API for working with SQLite. APSW and apswutils is used to
connect to SQLite, optimized for speed and clean error handling.

``` python
from fastlite import *
```

``` python
db = database(':memory:') # or database('data/app.db')
```

Tables are normally constructed with classes, field types are specified
as type hints.

``` python
class Book: isbn: str; title: str; pages: int; userid: int
# The transform arg instructs fastlite to change the db schema when fields change.
# Create only creates a table if the table doesn't exist.
books = db.create(Book, pk='isbn', transform=True)
                
class User: id: int; name: str; active: bool = True
# If no pk is provided, id is used as the primary key.
users = db.create(User, transform=True)
users
```

    <Table user (id, name, active)>

### Fastlite CRUD operations

Every operation in fastlite returns a full superset of dataclass
functionality.

``` python
user = users.insert(name='Alex',active=False)
user
```

    User(id=1, name='Alex', active=0)

``` python
# List all records
users()
```

    [User(id=1, name='Alex', active=0)]

``` python
# Limit, offset, and order results:
users(order_by='name', limit=2, offset=1)

# Filter on the results
users(where="name='Alex'")

# Placeholder for avoiding injection attacks
users("name=?", ('Alex',))

# A single record by pk
users[user.id]
```

    User(id=1, name='Alex', active=0)

Test if a record exists by using `in` keyword on primary key:

``` python
1 in users
```

    True

Updates (which take a dict or a typed object) return the updated record.

``` python
user.name='Lauren'
user.active=True
users.update(user)
```

    User(id=1, name='Lauren', active=1)

`.xtra()` to automatically constrain queries, updates, and inserts from
there on:

``` python
users.xtra(active=True)
users()
```

    [User(id=1, name='Lauren', active=1)]

Deleting by pk:

``` python
users.delete(user.id)
```

    <Table user (id, name, active)>

NotFoundError is raised by pk `[]`, updates, and deletes.

``` python
try: users['Amy']
except NotFoundError: print('User not found')
```

    User not found

## MonsterUI

MonsterUI is a shadcn-like component library for FastHTML. It adds the
Tailwind-based libraries FrankenUI and DaisyUI to FastHTML, as well as
Python’s mistletoe for Markdown, HighlightJS for code highlighting, and
Katex for latex support, following semantic HTML patterns when possible.
It is recommended for when you wish to go beyond the basics provided by
FastHTML’s built-in pico support.

A minimal app:

``` python
from fasthtml.common import *
from monsterui.all import *

app, rt = fast_app(hdrs=Theme.blue.headers(highlightjs=True)) # Use MonsterUI blue theme and highlight code in markdown

@rt
def index():
    socials = (('github','https://github.com/AnswerDotAI/MonsterUI'),)
    return Titled("App",
        Card(
            P("App", cls=TextPresets.muted_sm),
            # LabelInput, DivLAigned, and UkIconLink are non-semantic MonsterUI FT Components,
            LabelInput('Email', type='email', required=True),
            footer=DivLAligned(*[UkIconLink(icon,href=url) for icon,url in socials])))
```

MonsterUI recommendations:

- Use defaults as much as possible, for example
  [`Container`](https://www.fastht.ml/docs/api/pico.html#container) in
  monsterui already has defaults for margins
- Use `*T` for button styling consistency, for example
  `cls=ButtonT.destructive` for a red delete button or
  `cls=ButtonT.primary` for a CTA button
- Use `Label*` functions for forms as much as possible
  (e.g. `LabelInput`, `LabelRange`) which creates and links both the
  `FormLabel` and user input appropriately to avoid boiler plate.

Flex Layout Elements (such as `DivLAligned` and `DivFullySpaced`) can be
used to create layouts concisely

``` python
def TeamCard(name, role, location="Remote"):
    icons = ("mail", "linkedin", "github")
    return Card(
        DivLAligned(
            DiceBearAvatar(name, h=24, w=24),
            Div(H3(name), P(role))),
        footer=DivFullySpaced(
            DivHStacked(UkIcon("map-pin", height=16), P(location)),
            DivHStacked(*(UkIconLink(icon, height=16) for icon in icons))))
```

Forms are styled and spaced for you without significant additional
classes.

``` python
def MonsterForm():
    relationship = ["Parent",'Sibling', "Friend", "Spouse", "Significant Other", "Relative", "Child", "Other"]
    return Div(
        DivCentered(
            H3("Emergency Contact Form"),
            P("Please fill out the form completely", cls=TextPresets.muted_sm)),
        Form(
            Grid(LabelInput("Name",id='name'),LabelInput("Email",     id='email')),
            H3("Relationship to patient"),
            Grid(*[LabelCheckboxX(o) for o in relationship], cols=4, cls='space-y-3'),
            DivCentered(Button("Submit Form", cls=ButtonT.primary))),
        cls='space-y-4')
```

Text can be styled with markdown automatically with MonsterUI

```` python
render_md("""
# My Document

> Important note here

+ List item with **bold**
+ Another with `code`

```python
def hello():
    print("world")
```
""")
````

    '<div><h1 class="uk-h1 text-4xl font-bold mt-12 mb-6">My Document</h1>\n<blockquote class="uk-blockquote pl-4 border-l-4 border-primary italic mb-6">\n<p class="text-lg leading-relaxed mb-6">Important note here</p>\n</blockquote>\n<ul class="uk-list uk-list-bullet space-y-2 mb-6 ml-6 text-lg">\n<li class="leading-relaxed">List item with <strong>bold</strong></li>\n<li class="leading-relaxed">Another with <code class="uk-codespan px-1">code</code></li>\n</ul>\n<pre class="bg-base-200 rounded-lg p-4 mb-6"><code class="language-python uk-codespan px-1 uk-codespan px-1 block overflow-x-auto">def hello():\n    print("world")\n</code></pre>\n</div>'

Or using semantic HTML:

``` python
def SemanticText():
    return Card(
        H1("MonsterUI's Semantic Text"),
        P(
            Strong("MonsterUI"), " brings the power of semantic HTML to life with ",
            Em("beautiful styling"), " and ", Mark("zero configuration"), "."),
        Blockquote(
            P("Write semantic HTML in pure Python, get modern styling for free."),
            Cite("MonsterUI Team")),
        footer=Small("Released February 2025"),)
```</doc><doc title="HTMX reference" desc="Brief description of all HTMX attributes, CSS classes, headers, events, extensions, js lib methods, and config options">+++
title = "Reference"
+++

## Contents

* [htmx Core Attributes](#attributes)
* [htmx Additional Attributes](#attributes-additional)
* [htmx CSS Classes](#classes)
* [htmx Request Headers](#request_headers)
* [htmx Response Headers](#response_headers)
* [htmx Events](#events)
* [htmx Extensions](/extensions)
* [JavaScript API](#api)
* [Configuration Options](#config)

## Core Attribute Reference {#attributes}

The most common attributes when using htmx.

<div class="info-table">

| Attribute                                        | Description                                                                                                        |
|--------------------------------------------------|--------------------------------------------------------------------------------------------------------------------|
| [`hx-get`](@/attributes/hx-get.md)               | issues a `GET` to the specified URL                                                                                |
| [`hx-post`](@/attributes/hx-post.md)             | issues a `POST` to the specified URL                                                                               |
| [`hx-on*`](@/attributes/hx-on.md)                | handle events with inline scripts on elements                                                                      |
| [`hx-push-url`](@/attributes/hx-push-url.md)     | push a URL into the browser location bar to create history                                                         |
| [`hx-select`](@/attributes/hx-select.md)         | select content to swap in from a response                                                                          |
| [`hx-select-oob`](@/attributes/hx-select-oob.md) | select content to swap in from a response, somewhere other than the target (out of band)                           |
| [`hx-swap`](@/attributes/hx-swap.md)             | controls how content will swap in (`outerHTML`, `beforeend`, `afterend`, ...)                                      |
| [`hx-swap-oob`](@/attributes/hx-swap-oob.md)     | mark element to swap in from a response (out of band)                                                              |
| [`hx-target`](@/attributes/hx-target.md)         | specifies the target element to be swapped                                                                         |
| [`hx-trigger`](@/attributes/hx-trigger.md)       | specifies the event that triggers the request                                                                      |
| [`hx-vals`](@/attributes/hx-vals.md)             | add values to submit with the request (JSON format)                                                                |

</div>

## Additional Attribute Reference {#attributes-additional}

All other attributes available in htmx.

<div class="info-table">

| Attribute                                            | Description                                                                                                                        |
|------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------|
| [`hx-boost`](@/attributes/hx-boost.md)               | add [progressive enhancement](https://en.wikipedia.org/wiki/Progressive_enhancement) for links and forms                           |
| [`hx-confirm`](@/attributes/hx-confirm.md)           | shows a `confirm()` dialog before issuing a request                                                                                |
| [`hx-delete`](@/attributes/hx-delete.md)             | issues a `DELETE` to the specified URL                                                                                             |
| [`hx-disable`](@/attributes/hx-disable.md)           | disables htmx processing for the given node and any children nodes                                                                 |
| [`hx-disabled-elt`](@/attributes/hx-disabled-elt.md) | adds the `disabled` attribute to the specified elements while a request is in flight                                               |
| [`hx-disinherit`](@/attributes/hx-disinherit.md)     | control and disable automatic attribute inheritance for child nodes                                                                |
| [`hx-encoding`](@/attributes/hx-encoding.md)         | changes the request encoding type                                                                                                  |
| [`hx-ext`](@/attributes/hx-ext.md)                   | extensions to use for this element                                                                                                 |
| [`hx-headers`](@/attributes/hx-headers.md)           | adds to the headers that will be submitted with the request                                                                        |
| [`hx-history`](@/attributes/hx-history.md)           | prevent sensitive data being saved to the history cache                                                                            |
| [`hx-history-elt`](@/attributes/hx-history-elt.md)   | the element to snapshot and restore during history navigation                                                                      |
| [`hx-include`](@/attributes/hx-include.md)           | include additional data in requests                                                                                                |
| [`hx-indicator`](@/attributes/hx-indicator.md)       | the element to put the `htmx-request` class on during the request                                                                  |
| [`hx-inherit`](@/attributes/hx-inherit.md)           | control and enable automatic attribute inheritance for child nodes if it has been disabled by default                            |
| [`hx-params`](@/attributes/hx-params.md)             | filters the parameters that will be submitted with a request                                                                       |
| [`hx-patch`](@/attributes/hx-patch.md)               | issues a `PATCH` to the specified URL                                                                                              |
| [`hx-preserve`](@/attributes/hx-preserve.md)         | specifies elements to keep unchanged between requests                                                                              |
| [`hx-prompt`](@/attributes/hx-prompt.md)             | shows a `prompt()` before submitting a request                                                                                     |
| [`hx-put`](@/attributes/hx-put.md)                   | issues a `PUT` to the specified URL                                                                                                |
| [`hx-replace-url`](@/attributes/hx-replace-url.md)   | replace the URL in the browser location bar                                                                                        |
| [`hx-request`](@/attributes/hx-request.md)           | configures various aspects of the request                                                                                          |
| [`hx-sync`](@/attributes/hx-sync.md)                 | control how requests made by different elements are synchronized                                                                   |
| [`hx-validate`](@/attributes/hx-validate.md)         | force elements to validate themselves before a request                                                                             |
| [`hx-vars`](@/attributes/hx-vars.md)                 | adds values dynamically to the parameters to submit with the request (deprecated, please use [`hx-vals`](@/attributes/hx-vals.md)) |

</div>

## CSS Class Reference {#classes}

<div class="info-table">

| Class | Description |
|-----------|-------------|
| `htmx-added` | Applied to a new piece of content before it is swapped, removed after it is settled.
| `htmx-indicator` | A dynamically generated class that will toggle visible (opacity:1) when a `htmx-request` class is present
| `htmx-request` | Applied to either the element or the element specified with [`hx-indicator`](@/attributes/hx-indicator.md) while a request is ongoing
| `htmx-settling` | Applied to a target after content is swapped, removed after it is settled. The duration can be modified via [`hx-swap`](@/attributes/hx-swap.md).
| `htmx-swapping` | Applied to a target before any content is swapped, removed after it is swapped. The duration can be modified via [`hx-swap`](@/attributes/hx-swap.md).

</div>

## HTTP Header Reference {#headers}

### Request Headers Reference {#request_headers}

<div class="info-table">

| Header | Description |
|--------|-------------|
| `HX-Boosted` | indicates that the request is via an element using [hx-boost](@/attributes/hx-boost.md)
| `HX-Current-URL` | the current URL of the browser
| `HX-History-Restore-Request` | "true" if the request is for history restoration after a miss in the local history cache
| `HX-Prompt` | the user response to an [hx-prompt](@/attributes/hx-prompt.md)
| `HX-Request` | always "true"
| `HX-Target` | the `id` of the target element if it exists
| `HX-Trigger-Name` | the `name` of the triggered element if it exists
| `HX-Trigger` | the `id` of the triggered element if it exists

</div>

### Response Headers Reference {#response_headers}

<div class="info-table">

| Header                                               | Description |
|------------------------------------------------------|-------------|
| [`HX-Location`](@/headers/hx-location.md)            | allows you to do a client-side redirect that does not do a full page reload
| [`HX-Push-Url`](@/headers/hx-push-url.md)            | pushes a new url into the history stack
| [`HX-Redirect`](@/headers/hx-redirect.md)            | can be used to do a client-side redirect to a new location
| `HX-Refresh`                                         | if set to "true" the client-side will do a full refresh of the page
| [`HX-Replace-Url`](@/headers/hx-replace-url.md)      | replaces the current URL in the location bar
| `HX-Reswap`                                          | allows you to specify how the response will be swapped. See [hx-swap](@/attributes/hx-swap.md) for possible values
| `HX-Retarget`                                        | a CSS selector that updates the target of the content update to a different element on the page
| `HX-Reselect`                                        | a CSS selector that allows you to choose which part of the response is used to be swapped in. Overrides an existing [`hx-select`](@/attributes/hx-select.md) on the triggering element
| [`HX-Trigger`](@/headers/hx-trigger.md)              | allows you to trigger client-side events
| [`HX-Trigger-After-Settle`](@/headers/hx-trigger.md) | allows you to trigger client-side events after the settle step
| [`HX-Trigger-After-Swap`](@/headers/hx-trigger.md)   | allows you to trigger client-side events after the swap step

</div>

## Event Reference {#events}

<div class="info-table">

| Event | Description |
|-------|-------------|
| [`htmx:abort`](@/events.md#htmx:abort) | send this event to an element to abort a request
| [`htmx:afterOnLoad`](@/events.md#htmx:afterOnLoad) | triggered after an AJAX request has completed processing a successful response
| [`htmx:afterProcessNode`](@/events.md#htmx:afterProcessNode) | triggered after htmx has initialized a node
| [`htmx:afterRequest`](@/events.md#htmx:afterRequest)  | triggered after an AJAX request has completed
| [`htmx:afterSettle`](@/events.md#htmx:afterSettle)  | triggered after the DOM has settled
| [`htmx:afterSwap`](@/events.md#htmx:afterSwap)  | triggered after new content has been swapped in
| [`htmx:beforeCleanupElement`](@/events.md#htmx:beforeCleanupElement)  | triggered before htmx [disables](@/attributes/hx-disable.md) an element or removes it from the DOM
| [`htmx:beforeOnLoad`](@/events.md#htmx:beforeOnLoad)  | triggered before any response processing occurs
| [`htmx:beforeProcessNode`](@/events.md#htmx:beforeProcessNode) | triggered before htmx initializes a node
| [`htmx:beforeRequest`](@/events.md#htmx:beforeRequest)  | triggered before an AJAX request is made
| [`htmx:beforeSwap`](@/events.md#htmx:beforeSwap)  | triggered before a swap is done, allows you to configure the swap
| [`htmx:beforeSend`](@/events.md#htmx:beforeSend)  | triggered just before an ajax request is sent
| [`htmx:beforeTransition`](@/events.md#htmx:beforeTransition)  | triggered before the [View Transition](https://developer.mozilla.org/en-US/docs/Web/API/View_Transitions_API) wrapped swap occurs
| [`htmx:configRequest`](@/events.md#htmx:configRequest)  | triggered before the request, allows you to customize parameters, headers
| [`htmx:confirm`](@/events.md#htmx:confirm)  | triggered after a trigger occurs on an element, allows you to cancel (or delay) issuing the AJAX request
| [`htmx:historyCacheError`](@/events.md#htmx:historyCacheError)  | triggered on an error during cache writing
| [`htmx:historyCacheHit`](@/events.md#htmx:historyCacheHit)  | triggered on a cache hit in the history subsystem
| [`htmx:historyCacheMiss`](@/events.md#htmx:historyCacheMiss)  | triggered on a cache miss in the history subsystem
| [`htmx:historyCacheMissLoadError`](@/events.md#htmx:historyCacheMissLoadError)  | triggered on a unsuccessful remote retrieval
| [`htmx:historyCacheMissLoad`](@/events.md#htmx:historyCacheMissLoad)  | triggered on a successful remote retrieval
| [`htmx:historyRestore`](@/events.md#htmx:historyRestore)  | triggered when htmx handles a history restoration action
| [`htmx:beforeHistorySave`](@/events.md#htmx:beforeHistorySave)  | triggered before content is saved to the history cache
| [`htmx:load`](@/events.md#htmx:load)  | triggered when new content is added to the DOM
| [`htmx:noSSESourceError`](@/events.md#htmx:noSSESourceError)  | triggered when an element refers to a SSE event in its trigger, but no parent SSE source has been defined
| [`htmx:onLoadError`](@/events.md#htmx:onLoadError)  | triggered when an exception occurs during the onLoad handling in htmx
| [`htmx:oobAfterSwap`](@/events.md#htmx:oobAfterSwap)  | triggered after an out of band element as been swapped in
| [`htmx:oobBeforeSwap`](@/events.md#htmx:oobBeforeSwap)  | triggered before an out of band element swap is done, allows you to configure the swap
| [`htmx:oobErrorNoTarget`](@/events.md#htmx:oobErrorNoTarget)  | triggered when an out of band element does not have a matching ID in the current DOM
| [`htmx:prompt`](@/events.md#htmx:prompt)  | triggered after a prompt is shown
| [`htmx:pushedIntoHistory`](@/events.md#htmx:pushedIntoHistory)  | triggered after a url is pushed into history
| [`htmx:replacedInHistory`](@/events.md#htmx:replacedInHistory)  | triggered after a url is replaced in history
| [`htmx:responseError`](@/events.md#htmx:responseError)  | triggered when an HTTP response error (non-`200` or `300` response code) occurs
| [`htmx:sendAbort`](@/events.md#htmx:sendAbort)  | triggered when a request is aborted
| [`htmx:sendError`](@/events.md#htmx:sendError)  | triggered when a network error prevents an HTTP request from happening
| [`htmx:sseError`](@/events.md#htmx:sseError)  | triggered when an error occurs with a SSE source
| [`htmx:sseOpen`](/events#htmx:sseOpen)  | triggered when a SSE source is opened
| [`htmx:swapError`](@/events.md#htmx:swapError)  | triggered when an error occurs during the swap phase
| [`htmx:targetError`](@/events.md#htmx:targetError)  | triggered when an invalid target is specified
| [`htmx:timeout`](@/events.md#htmx:timeout)  | triggered when a request timeout occurs
| [`htmx:validation:validate`](@/events.md#htmx:validation:validate)  | triggered before an element is validated
| [`htmx:validation:failed`](@/events.md#htmx:validation:failed)  | triggered when an element fails validation
| [`htmx:validation:halted`](@/events.md#htmx:validation:halted)  | triggered when a request is halted due to validation errors
| [`htmx:xhr:abort`](@/events.md#htmx:xhr:abort)  | triggered when an ajax request aborts
| [`htmx:xhr:loadend`](@/events.md#htmx:xhr:loadend)  | triggered when an ajax request ends
| [`htmx:xhr:loadstart`](@/events.md#htmx:xhr:loadstart)  | triggered when an ajax request starts
| [`htmx:xhr:progress`](@/events.md#htmx:xhr:progress)  | triggered periodically during an ajax request that supports progress events

</div>

## JavaScript API Reference {#api}

<div class="info-table">

| Method | Description |
|-------|-------------|
| [`htmx.addClass()`](@/api.md#addClass)  | Adds a class to the given element
| [`htmx.ajax()`](@/api.md#ajax)  | Issues an htmx-style ajax request
| [`htmx.closest()`](@/api.md#closest)  | Finds the closest parent to the given element matching the selector
| [`htmx.config`](@/api.md#config)  | A property that holds the current htmx config object
| [`htmx.createEventSource`](@/api.md#createEventSource)  | A property holding the function to create SSE EventSource objects for htmx
| [`htmx.createWebSocket`](@/api.md#createWebSocket)  | A property holding the function to create WebSocket objects for htmx
| [`htmx.defineExtension()`](@/api.md#defineExtension)  | Defines an htmx [extension](https://htmx.org/extensions)
| [`htmx.find()`](@/api.md#find)  | Finds a single element matching the selector
| [`htmx.findAll()` `htmx.findAll(elt, selector)`](@/api.md#find)  | Finds all elements matching a given selector
| [`htmx.logAll()`](@/api.md#logAll)  | Installs a logger that will log all htmx events
| [`htmx.logger`](@/api.md#logger)  | A property set to the current logger (default is `null`)
| [`htmx.off()`](@/api.md#off)  | Removes an event listener from the given element
| [`htmx.on()`](@/api.md#on)  | Creates an event listener on the given element, returning it
| [`htmx.onLoad()`](@/api.md#onLoad)  | Adds a callback handler for the `htmx:load` event
| [`htmx.parseInterval()`](@/api.md#parseInterval)  | Parses an interval declaration into a millisecond value
| [`htmx.process()`](@/api.md#process)  | Processes the given element and its children, hooking up any htmx behavior
| [`htmx.remove()`](@/api.md#remove)  | Removes the given element
| [`htmx.removeClass()`](@/api.md#removeClass)  | Removes a class from the given element
| [`htmx.removeExtension()`](@/api.md#removeExtension)  | Removes an htmx [extension](https://htmx.org/extensions)
| [`htmx.swap()`](@/api.md#swap)  | Performs swapping (and settling) of HTML content
| [`htmx.takeClass()`](@/api.md#takeClass)  | Takes a class from other elements for the given element
| [`htmx.toggleClass()`](@/api.md#toggleClass)  | Toggles a class from the given element
| [`htmx.trigger()`](@/api.md#trigger)  | Triggers an event on an element
| [`htmx.values()`](@/api.md#values)  | Returns the input values associated with the given element

</div>


## Configuration Reference {#config}

Htmx has some configuration options that can be accessed either programmatically or declaratively.  They are
listed below:

<div class="info-table">

| Config Variable                        | Info                                                                                                                                                                       |
|----------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `htmx.config.historyEnabled`           | defaults to `true`, really only useful for testing                                                                                                                         |
| `htmx.config.historyCacheSize`         | defaults to 10                                                                                                                                                             |
| `htmx.config.refreshOnHistoryMiss`     | defaults to `false`, if set to `true` htmx will issue a full page refresh on history misses rather than use an AJAX request                                                |
| `htmx.config.defaultSwapStyle`         | defaults to `innerHTML`                                                                                                                                                    |
| `htmx.config.defaultSwapDelay`         | defaults to 0                                                                                                                                                              |
| `htmx.config.defaultSettleDelay`       | defaults to 20                                                                                                                                                             |
| `htmx.config.includeIndicatorStyles`   | defaults to `true` (determines if the indicator styles are loaded)                                                                                                         |
| `htmx.config.indicatorClass`           | defaults to `htmx-indicator`                                                                                                                                               |
| `htmx.config.requestClass`             | defaults to `htmx-request`                                                                                                                                                 |
| `htmx.config.addedClass`               | defaults to `htmx-added`                                                                                                                                                   |
| `htmx.config.settlingClass`            | defaults to `htmx-settling`                                                                                                                                                |
| `htmx.config.swappingClass`            | defaults to `htmx-swapping`                                                                                                                                                |
| `htmx.config.allowEval`                | defaults to `true`, can be used to disable htmx's use of eval for certain features (e.g. trigger filters)                                                                  |
| `htmx.config.allowScriptTags`          | defaults to `true`, determines if htmx will process script tags found in new content                                                                                       |
| `htmx.config.inlineScriptNonce`        | defaults to `''`, meaning that no nonce will be added to inline scripts                                                                                                    |
| `htmx.config.inlineStyleNonce`         | defaults to `''`, meaning that no nonce will be added to inline styles                                                                                                     |
| `htmx.config.attributesToSettle`       | defaults to `["class", "style", "width", "height"]`, the attributes to settle during the settling phase                                                                    |
| `htmx.config.wsReconnectDelay`         | defaults to `full-jitter`                                                                                                                                                  |
| `htmx.config.wsBinaryType`             | defaults to `blob`, the [the type of binary data](https://developer.mozilla.org/docs/Web/API/WebSocket/binaryType) being received over the WebSocket connection            |
| `htmx.config.disableSelector`          | defaults to `[hx-disable], [data-hx-disable]`, htmx will not process elements with this attribute on it or a parent                                                        |
| `htmx.config.disableInheritance`       | defaults to `false`. If it is set to `true`, the inheritance of attributes is completely disabled and you can explicitly specify the inheritance with the [hx-inherit](@/attributes/hx-inherit.md) attribute.
| `htmx.config.withCredentials`          | defaults to `false`, allow cross-site Access-Control requests using credentials such as cookies, authorization headers or TLS client certificates                          |
| `htmx.config.timeout`                  | defaults to 0, the number of milliseconds a request can take before automatically being terminated                                                                         |
| `htmx.config.scrollBehavior`           | defaults to 'instant', the scroll behavior when using the [show](@/attributes/hx-swap.md#scrolling-scroll-show) modifier with `hx-swap`. The allowed values are `instant` (scrolling should happen instantly in a single jump), `smooth` (scrolling should animate smoothly) and `auto` (scroll behavior is determined by the computed value of [scroll-behavior](https://developer.mozilla.org/en-US/docs/Web/CSS/scroll-behavior)). |
| `htmx.config.defaultFocusScroll`       | if the focused element should be scrolled into view, defaults to false and can be overridden using the [focus-scroll](@/attributes/hx-swap.md#focus-scroll) swap modifier. |
| `htmx.config.getCacheBusterParam`      | defaults to false, if set to true htmx will append the target element to the `GET` request in the format `org.htmx.cache-buster=targetElementId`                           |
| `htmx.config.globalViewTransitions`    | if set to `true`, htmx will use the [View Transition](https://developer.mozilla.org/en-US/docs/Web/API/View_Transitions_API) API when swapping in new content.             |
| `htmx.config.methodsThatUseUrlParams`  | defaults to `["get", "delete"]`, htmx will format requests with these methods by encoding their parameters in the URL, not the request body                                |
| `htmx.config.selfRequestsOnly`         | defaults to `true`, whether to only allow AJAX requests to the same domain as the current document                                                             |
| `htmx.config.ignoreTitle`              | defaults to `false`, if set to `true` htmx will not update the title of the document when a `title` tag is found in new content                                            |
| `htmx.config.scrollIntoViewOnBoost`    | defaults to `true`, whether or not the target of a boosted element is scrolled into the viewport. If `hx-target` is omitted on a boosted element, the target defaults to `body`, causing the page to scroll to the top. |
| `htmx.config.triggerSpecsCache`        | defaults to `null`, the cache to store evaluated trigger specifications into, improving parsing performance at the cost of more memory usage. You may define a simple object to use a never-clearing cache, or implement your own system using a [proxy object](https://developer.mozilla.org/docs/Web/JavaScript/Reference/Global_Objects/Proxy) |
| `htmx.config.responseHandling`         | the default [Response Handling](@/docs.md#response-handling) behavior for response status codes can be configured here to either swap or error                             |
| `htmx.config.allowNestedOobSwaps`      | defaults to `true`, whether to process OOB swaps on elements that are nested within the main response element. See [Nested OOB Swaps](@/attributes/hx-swap-oob.md#nested-oob-swaps). |
| `htmx.config.historyRestoreAsHxRequest`| defaults to `true`, Whether to treat history cache miss full page reload requests as a "HX-Request" by returning this response header. This should always be disabled when using HX-Request header to optionally return partial responses                                                                                                         |


</div>

You can set them directly in javascript, or you can use a `meta` tag:

```html
<meta name="htmx-config" content='{"defaultSwapStyle":"outerHTML"}'>
```</doc><doc title="Starlette quick guide" desc="A quick overview of some Starlette features useful to FastHTML devs."># 🌟 Starlette Quick Manual


2020-02-09

Starlette is the ASGI web framework used as the foundation of FastHTML. Listed here are some Starlette features FastHTML developers can use directly, since the `FastHTML` class inherits from the `Starlette` class (but note that FastHTML has its own customised `RouteX` and `RouterX` classes for routing, to handle FT element trees etc).

## Get uploaded file content

```
async def handler(request):
    inp = await request.form()
    uploaded_file = inp["filename"]
    filename = uploaded_file.filename           # abc.png
    content_type = uploaded.content_type    # MIME type, e.g. image/png
    content = await uploaded_file.read()       # image content

```

## Return a customized response (status code and headers)

```
import json
from starlette.responses import Response

async def handler(request):
    data = {
        "name": "Bo"
    }
    return Response(json.dumps(data), media_type="application/json")

```

`Response` takes `status_code`, `headers` and `media_type`, so if we want to change a response's status code, we can do:

```
return Response(content, statu_code=404)

```

And customized headers:

```
headers = {
	"x-extra-key": "value"
}
return Response(content, status_code=200, headers=headers)

```

## Redirect

```
from starlette.responses import RedirectResponse

async handler(request):
    # Customize status_code: 
    #   301: permanent redirect 
    #   302: temporary redirect 
    #   303: see others
    #   307: temporary redirect (default)
    return RedirectResponse(url=url, status_code=303)

```

## Request context

### URL Object: `request.url`

  * Get request full url: `url = str(request.url)`
  * Get scheme: `request.url.scheme` (http, https, ws, wss)
  * Get netloc: `request.url.netloc`, e.g.: example.com:8080
  * Get path: `request.url.path`, e.g.: /search
  * Get query string: `request.url.query`, e.g.: kw=hello
  * Get hostname: `request.url.hostname`, e.g.: example.com
  * Get port: `request.url.port`, e.g.: 8080
  * If using secure scheme: `request.url.is_secure`, True is schme is `https` or `wss`

### Headers: `request.headers`

```
{
    'host': 'example.com:8080', 
    'connection': 'keep-alive', 
    'cache-control': 'max-age=0', 
    'sec-ch-ua': 'Google Chrome 80', 
    'dnt': '1', 
    'upgrade-insecure-requests': '1', 
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) ...',
    'sec-fetch-dest': 'document', 
    'accept': 'text/html,image/apng,*/*;q=0.8;v=b3;q=0.9', 
    'sec-origin-policy': '0', 
    'sec-fetch-site': 'none', 
    'sec-fetch-mode': 'navigate', 
    'sec-fetch-user': '?1', 
    'accept-encoding': 'gzip, deflate, br', 
    'accept-language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7,zh-TW;q=0.6', 
    'cookie': 'session=eyJhZG1pbl91c2_KiQ...'
}

```

### Client: `request.client`

  * `request.client.host`: get client sock IP
  * `request.client.port`: get client sock port

### Method: `request.method`

  * `request.method`: GET, POST, etc.

### Get Data

  * `await request.body()`: get raw data from body
  * `await request.json()`: get passed data and parse it as JSON
  * `await request.form()`: get posted data and pass it as dictionary

### Scope: `request.scope`

```
{
    'type': 'http', 
    'http_version': '1.1', 
    'server': ('127.0.0.1', 9092), 
    'client': ('127.0.0.1', 53102), 
    'scheme': 'https', 
    'method': 'GET', 
    'root_path': '', 
    'path': '/', 
    'raw_path': b'/', 
    'query_string': b'kw=hello', 
    'headers': [
        (b'host', b'example.com:8080'), 
        (b'connection', b'keep-alive'), 
        (b'cache-control', b'max-age=0'), 
        ...
    ], 
    'app': <starlette.applications.Starlette object at 0x1081bd650>, 
    'session': {'uid': '57ba03ea7333f72a25f837cf'}, 
    'router': <starlette.routing.Router object at 0x1081bd6d0>, 
    'endpoint': <class 'app.index.Index'>, 
    'path_params': {}
}

```

## Put varaible in request & app scope

```
app.state.dbconn = get_db_conn()
request.state.start_time = time.time()
# use app-scope state variable in a request
request.app.state.dbconn

```

## Utility functions

### Use `State` to wrap a dictionary

```
from starlette.datastructures import State

data = {
    "name": "Bo"
}
print(data["name"])
# now wrap it with State function
wrapped = State(data)
# You can use the dot syntaxt, but can't use `wrapped["name"]` any more.
print(wrapped.name)

```

### login_required wrapper function

NB: This is easier to do in FastHTML using Beforeware.

```
import functools
from starlette.endpoints import HTTPEndpoint
from starlette.responses import Response

def login_required(login_url="/signin"):
    def decorator(handler):
        @functools.wraps(handler)
        async def new_handler(obj, req, *args, **kwargs):
            user = req.session.get("login_user")
            if user is None:
                return seeother(login_url)
            return await handler(obj, req, *args, **kwargs)
        return new_handler
    return decorator

class MyAccount(HTTPEndpiont):
    @login_required()
    async def get(self, request):
        # some logic here
        content = "hello"
        return Response(content)

```

## Exceptions

Handle exception and customize 403, 404, 503, 500 page:

```
from starlette.exceptions import HTTPException

async def exc_handle_403(request, exc):
    return HTMLResponse("My 403 page", status_code=exc.status_code)

async def exc_handle_404(request, exc):
    return HTMLResponse("My 404 page", status_code=exc.status_code)

async def exc_handle_503(request, exc):
    return HTMLResponse("Failed, please try it later", status_code=exc.status_code)

# error is not exception, 500 is server side unexpected error, all other status code will be treated as Exception
async def err_handle_500(request, exc):
    import traceback
    Log.error(traceback.format_exc())
    return HTMLResponse("My 500 page", status_code=500)

# To add handler, we can add either status_code or Exception itself as key
exception_handlers = {
    403: exc_handle_403,
    404: exc_handle_404,
    503: exc_handle_503,
    500: err_handle_500,
    #HTTPException: exc_handle_500,
}

app = Starlette(routes=routes, exception_handlers=exception_handlers)

```

## Background Task

### Put some async task as background task

```
import aiofiles
from starlette.background import BackgroundTask
from starlette.responses import Response

aiofiles_remove = aiofiles.os.wrap(os.remove)

async def del_file(fpath):
    await aiofiles_remove(fpath)

async def handler(request):
    content = ""
    fpath = "/tmp/tmpfile.txt"
    task = BackgroundTask(del_file, fpath=fpath)
    return Response(content, background=task)

```

### Put multiple tasks as background task

```
from starlette.background import BackgroundTasks

async def task1(name):
    pass

async def task2(email):
    pass

async def handler(request):
    tasks = BackgroundTasks()
    tasks.add_task(task1, name="John")
    tasks.add_task(task2, email="info@example.com")
    content = ""
    return Response(content, background=tasks)

```

## Write middleware

There are 2 ways to write middleware:

### Define `__call__` function:

```
class MyMiddleware:
    def __init__(self, app):
        self.app = app

    async def __call__(self, scope, receive, send):
        # see above scope dictionary as reference
        headers = dict(scope["headers"])
        # do something
        # pass to next middleware
        return await self.app(scope, receive, send)

```

### Use `BaseHTTPMiddleware`

```
from starlette.middleware.base import BaseHTTPMiddleware

class CustomHeaderMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request, call_next):
        # do something before pass to next middleware
        response = await call_next(request)
        # do something after next middleware returned
        response.headers['X-Author'] = 'John'
        return response

```</doc></docs><api><doc title="API List" desc="A succint list of all functions and methods in fasthtml."># fasthtml Module Documentation

## fasthtml.authmw

- `class BasicAuthMiddleware`
    - `def __init__(self, app, cb, skip)`
    - `def __call__(self, scope, receive, send)`
    - `def authenticate(self, conn)`

## fasthtml.cli

- `@call_parse def railway_link()`
    Link the current directory to the current project's Railway service

- `@call_parse def railway_deploy(name, mount)`
    Deploy a FastHTML app to Railway

## fasthtml.components

> `ft_html` and `ft_hx` functions to add some conveniences to `ft`, along with a full set of basic HTML components, and functions to work with forms and `FT` conversion

- `def File(fname)`
    Use the unescaped text in file `fname` directly

- `def show(ft, *rest)`
    Renders FT Components into HTML within a Jupyter notebook.

- `def fill_form(form, obj)`
    Fills named items in `form` using attributes in `obj`

- `def fill_dataclass(src, dest)`
    Modifies dataclass in-place and returns it

- `def find_inputs(e, tags, **kw)`
    Recursively find all elements in `e` with `tags` and attrs matching `kw`

- `def html2ft(html, attr1st)`
    Convert HTML to an `ft` expression

- `def sse_message(elm, event)`
    Convert element `elm` into a format suitable for SSE streaming

## fasthtml.core

> The `FastHTML` subclass of `Starlette`, along with the `RouterX` and `RouteX` classes it automatically uses.

- `def parsed_date(s)`
    Convert `s` to a datetime

- `def snake2hyphens(s)`
    Convert `s` from snake case to hyphenated and capitalised

- `@dataclass class HtmxHeaders`
    - `def __bool__(self)`
    - `def __init__(self, boosted, current_url, history_restore_request, prompt, request, target, trigger_name, trigger)`

- `@dataclass class HttpHeader`
    - `def __init__(self, k, v)`

- `@use_kwargs_dict(**htmx_resps) def HtmxResponseHeaders(**kwargs)`
    HTMX response headers

- `def form2dict(form)`
    Convert starlette form data to a dict

- `def parse_form(req)`
    Starlette errors on empty multipart forms, so this checks for that situation

- `class JSONResponse`
    Same as starlette's version, but auto-stringifies non serializable types

    - `def render(self, content)`

- `def flat_xt(lst)`
    Flatten lists

- `class Beforeware`
    - `def __init__(self, f, skip)`

- `def EventStream(s)`
    Create a text/event-stream response from `s`

- `def flat_tuple(o)`
    Flatten lists

- `def noop_body(c, req)`
    Default Body wrap function which just returns the content

- `def respond(req, heads, bdy)`
    Default FT response creation function

- `class Redirect`
    Use HTMX or Starlette RedirectResponse as required to redirect to `loc`

    - `def __init__(self, loc)`
    - `def __response__(self, req)`

- `def qp(p, **kw)`
    Add parameters kw to path p

- `def def_hdrs(htmx, surreal)`
    Default headers for a FastHTML app

- `class FastHTML`
    - `def __init__(self, debug, routes, middleware, title, exception_handlers, on_startup, on_shutdown, lifespan, hdrs, ftrs, exts, before, after, surreal, htmx, default_hdrs, sess_cls, secret_key, session_cookie, max_age, sess_path, same_site, sess_https_only, sess_domain, key_fname, body_wrap, htmlkw, nb_hdrs, canonical, **bodykw)`

- `@patch def ws(self, path, conn, disconn, name, middleware)`
    Add a websocket route at `path`

- `def nested_name(f)`
    Get name of function `f` using '_' to join nested function names

- `@patch def route(self, path, methods, name, include_in_schema, body_wrap)`
    Add a route at `path`

- `def serve(appname, app, host, port, reload, reload_includes, reload_excludes)`
    Run the app in an async server, with live reload set as the default.

- `class Client`
    A simple httpx ASGI client that doesn't require `async`

    - `def __init__(self, app, url)`

- `class RouteFuncs`
    - `def __init__(self)`
    - `def __setattr__(self, name, value)`
    - `def __getattr__(self, name)`
    - `def __dir__(self)`

- `class APIRouter`
    Add routes to an app

    - `def __init__(self, prefix, body_wrap)`
    - `def __call__(self, path, methods, name, include_in_schema, body_wrap)`
        Add a route at `path`

    - `def __getattr__(self, name)`
    - `def to_app(self, app)`
        Add routes to `app`

    - `def ws(self, path, conn, disconn, name, middleware)`
        Add a websocket route at `path`


- `def cookie(key, value, max_age, expires, path, domain, secure, httponly, samesite)`
    Create a 'set-cookie' `HttpHeader`

- `@patch def static_route_exts(self, prefix, static_path, exts)`
    Add a static route at URL path `prefix` with files from `static_path` and `exts` defined by `reg_re_param()`

- `@patch def static_route(self, ext, prefix, static_path)`
    Add a static route at URL path `prefix` with files from `static_path` and single `ext` (including the '.')

- `class MiddlewareBase`
    - `def __call__(self, scope, receive, send)`

- `class FtResponse`
    Wrap an FT response with any Starlette `Response`

    - `def __init__(self, content, status_code, headers, cls, media_type, background)`
    - `def __response__(self, req)`

## fasthtml.fastapp

> The `fast_app` convenience wrapper

- `def fast_app(db_file, render, hdrs, ftrs, tbls, before, middleware, live, debug, title, routes, exception_handlers, on_startup, on_shutdown, lifespan, default_hdrs, pico, surreal, htmx, exts, canonical, secret_key, key_fname, session_cookie, max_age, sess_path, same_site, sess_https_only, sess_domain, htmlkw, bodykw, reload_attempts, reload_interval, static_path, body_wrap, nb_hdrs, **kwargs)`
    Create a FastHTML or FastHTMLWithLiveReload app.

## fasthtml.js

> Basic external Javascript lib wrappers

- `def light_media(css)`
    Render light media for day mode views

- `def dark_media(css)`
    Render dark media for night mode views

- `def MarkdownJS(sel)`
    Implements browser-based markdown rendering.

- `def HighlightJS(sel, langs, light, dark)`
    Implements browser-based syntax highlighting. Usage example [here](/tutorials/quickstart_for_web_devs.html#code-highlighting).

- `def MermaidJS(sel, theme)`
    Implements browser-based Mermaid diagram rendering.

## fasthtml.jupyter

> Use FastHTML in Jupyter notebooks

- `def nb_serve(app, log_level, port, host, **kwargs)`
    Start a Jupyter compatible uvicorn server with ASGI `app` on `port` with `log_level`

- `def nb_serve_async(app, log_level, port, host, **kwargs)`
    Async version of `nb_serve`

- `def is_port_free(port, host)`
    Check if `port` is free on `host`

- `def wait_port_free(port, host, max_wait)`
    Wait for `port` to be free on `host`

- `class JupyUvi`
    Start and stop a Jupyter compatible uvicorn server with ASGI `app` on `port` with `log_level`

    - `def __init__(self, app, log_level, host, port, start, **kwargs)`
    - `def start(self)`
    - `def start_async(self)`
    - `def stop(self)`

- `class JupyUviAsync`
    Start and stop an async Jupyter compatible uvicorn server with ASGI `app` on `port` with `log_level`

    - `def __init__(self, app, log_level, host, port, **kwargs)`
    - `def start(self)`
    - `def stop(self)`

- `def HTMX(path, host, app, port, height, link, iframe)`
    An iframe which displays the HTMX application in a notebook.

## fasthtml.live_reload

- `class FastHTMLWithLiveReload`
    `FastHTMLWithLiveReload` enables live reloading.
    This means that any code changes saved on the server will automatically
    trigger a reload of both the server and browser window.

    How does it work?
      - a websocket is created at `/live-reload`
      - a small js snippet `LIVE_RELOAD_SCRIPT` is injected into each webpage
      - this snippet connects to the websocket at `/live-reload` and listens for an `onclose` event
      - when the `onclose` event is detected the browser is reloaded

    Why do we listen for an `onclose` event?
      When code changes are saved the server automatically reloads if the --reload flag is set.
      The server reload kills the websocket connection. The `onclose` event serves as a proxy
      for "developer has saved some changes".

    Usage
        >>> from fasthtml.common import *
        >>> app = FastHTMLWithLiveReload()

        Run:
            serve()

    - `def __init__(self, *args, **kwargs)`

## fasthtml.oauth

> Basic scaffolding for handling OAuth

- `class GoogleAppClient`
    A `WebApplicationClient` for Google oauth2

    - `def __init__(self, client_id, client_secret, code, scope, project_id, **kwargs)`
    - `@classmethod def from_file(cls, fname, code, scope, **kwargs)`

- `class GitHubAppClient`
    A `WebApplicationClient` for GitHub oauth2

    - `def __init__(self, client_id, client_secret, code, scope, **kwargs)`

- `class HuggingFaceClient`
    A `WebApplicationClient` for HuggingFace oauth2

    - `def __init__(self, client_id, client_secret, code, scope, state, **kwargs)`

- `class DiscordAppClient`
    A `WebApplicationClient` for Discord oauth2

    - `def __init__(self, client_id, client_secret, is_user, perms, scope, **kwargs)`
    - `def login_link(self, redirect_uri, scope, state)`
    - `def parse_response(self, code, redirect_uri)`

- `class Auth0AppClient`
    A `WebApplicationClient` for Auth0 OAuth2

    - `def __init__(self, domain, client_id, client_secret, code, scope, redirect_uri, **kwargs)`
    - `def login_link(self, req)`

- `@patch def login_link(self, redirect_uri, scope, state, **kwargs)`
    Get a login link for this client

- `def redir_url(request, redir_path, scheme)`
    Get the redir url for the host in `request`

- `@patch def parse_response(self, code, redirect_uri)`
    Get the token from the oauth2 server response

- `@patch def get_info(self, token)`
    Get the info for authenticated user

- `@patch def retr_info(self, code, redirect_uri)`
    Combines `parse_response` and `get_info`

- `@patch def retr_id(self, code, redirect_uri)`
    Call `retr_info` and then return id/subscriber value

- `class OAuth`
    - `def __init__(self, app, cli, skip, redir_path, error_path, logout_path, login_path, https, http_patterns)`
    - `def redir_login(self, session)`
    - `def redir_url(self, req)`
    - `def login_link(self, req, scope, state)`
    - `def check_invalid(self, req, session, auth)`
    - `def logout(self, session)`
    - `def get_auth(self, info, ident, session, state)`

- `@patch() def consent_url(self, proj)`
    Get Google OAuth consent screen URL

- `@patch def save(self, fname)`
    Save credentials to `fname`

- `def load_creds(fname)`
    Load credentials from `fname`

- `@patch def creds(self)`
    Create `Credentials` from the client, refreshing if needed

## fasthtml.pico

> Basic components for generating Pico CSS tags

- `@delegates(ft_hx, keep=True) def Card(*c, **kwargs)`
    A PicoCSS Card, implemented as an Article with optional Header and Footer

- `@delegates(ft_hx, keep=True) def Group(*c, **kwargs)`
    A PicoCSS Group, implemented as a Fieldset with role 'group'

- `@delegates(ft_hx, keep=True) def Search(*c, **kwargs)`
    A PicoCSS Search, implemented as a Form with role 'search'

- `@delegates(ft_hx, keep=True) def Grid(*c, **kwargs)`
    A PicoCSS Grid, implemented as child Divs in a Div with class 'grid'

- `@delegates(ft_hx, keep=True) def DialogX(*c, **kwargs)`
    A PicoCSS Dialog, with children inside a Card

- `@delegates(ft_hx, keep=True) def Container(*args, **kwargs)`
    A PicoCSS Container, implemented as a Main with class 'container'

## fasthtml.stripe_otp

- `def create_price(app_nm, amt, currency)`
    Create a product and bind it to a price object. If product already exist just return the price list.

- `def archive_price(app_nm)`
    Archive a price - useful for cleanup if testing.

- `class Payment`

## fasthtml.svg

> Simple SVG FT elements

- `def Svg(*args, **kwargs)`
    An SVG tag; xmlns is added automatically, and viewBox defaults to height and width if not provided

- `@delegates(ft_hx) def ft_svg(tag, *c, **kwargs)`
    Create a standard `FT` element with some SVG-specific attrs

- `@delegates(ft_svg) def Rect(width, height, x, y, fill, stroke, stroke_width, rx, ry, **kwargs)`
    A standard SVG `rect` element

- `@delegates(ft_svg) def Circle(r, cx, cy, fill, stroke, stroke_width, **kwargs)`
    A standard SVG `circle` element

- `@delegates(ft_svg) def Ellipse(rx, ry, cx, cy, fill, stroke, stroke_width, **kwargs)`
    A standard SVG `ellipse` element

- `def transformd(translate, scale, rotate, skewX, skewY, matrix)`
    Create an SVG `transform` kwarg dict

- `@delegates(ft_svg) def Line(x1, y1, x2, y2, stroke, w, stroke_width, **kwargs)`
    A standard SVG `line` element

- `@delegates(ft_svg) def Polyline(*args, **kwargs)`
    A standard SVG `polyline` element

- `@delegates(ft_svg) def Polygon(*args, **kwargs)`
    A standard SVG `polygon` element

- `@delegates(ft_svg) def Text(*args, **kwargs)`
    A standard SVG `text` element

- `class PathFT`
    - `def M(self, x, y)`
        Move to.

    - `def L(self, x, y)`
        Line to.

    - `def H(self, x)`
        Horizontal line to.

    - `def V(self, y)`
        Vertical line to.

    - `def Z(self)`
        Close path.

    - `def C(self, x1, y1, x2, y2, x, y)`
        Cubic Bézier curve.

    - `def S(self, x2, y2, x, y)`
        Smooth cubic Bézier curve.

    - `def Q(self, x1, y1, x, y)`
        Quadratic Bézier curve.

    - `def T(self, x, y)`
        Smooth quadratic Bézier curve.

    - `def A(self, rx, ry, x_axis_rotation, large_arc_flag, sweep_flag, x, y)`
        Elliptical Arc.


- `def SvgOob(*args, **kwargs)`
    Wraps an SVG shape as required for an HTMX OOB swap

- `def SvgInb(*args, **kwargs)`
    Wraps an SVG shape as required for an HTMX inband swap

## fasthtml.xtend

> Simple extensions to standard HTML components, such as adding sensible defaults

- `@delegates(ft_hx, keep=True) def A(*c, **kwargs)`
    An A tag; `href` defaults to '#' for more concise use with HTMX

- `@delegates(ft_hx, keep=True) def AX(txt, hx_get, target_id, hx_swap, href, **kwargs)`
    An A tag with just one text child, allowing hx_get, target_id, and hx_swap to be positional params

- `@delegates(ft_hx, keep=True) def Form(*c, **kwargs)`
    A Form tag; identical to plain `ft_hx` version except default `enctype='multipart/form-data'`

- `@delegates(ft_hx, keep=True) def Hidden(value, id, **kwargs)`
    An Input of type 'hidden'

- `@delegates(ft_hx, keep=True) def CheckboxX(checked, label, value, id, name, **kwargs)`
    A Checkbox optionally inside a Label, preceded by a `Hidden` with matching name

- `@delegates(ft_html, keep=True) def Script(code, **kwargs)`
    A Script tag that doesn't escape its code

- `@delegates(ft_html, keep=True) def Style(*c, **kwargs)`
    A Style tag that doesn't escape its code

- `def double_braces(s)`
    Convert single braces to double braces if next to special chars or newline

- `def undouble_braces(s)`
    Convert double braces to single braces if next to special chars or newline

- `def loose_format(s, **kw)`
    String format `s` using `kw`, without being strict about braces outside of template params

- `def ScriptX(fname, src, nomodule, type, _async, defer, charset, crossorigin, integrity, **kw)`
    A `script` element with contents read from `fname`

- `def replace_css_vars(css, pre, **kwargs)`
    Replace `var(--)` CSS variables with `kwargs` if name prefix matches `pre`

- `def StyleX(fname, **kw)`
    A `style` element with contents read from `fname` and variables replaced from `kw`

- `def Nbsp()`
    A non-breaking space

- `def Surreal(code)`
    Wrap `code` in `domReadyExecute` and set `m=me()` and `p=me('-')`

- `def On(code, event, sel, me)`
    An async surreal.js script block event handler for `event` on selector `sel,p`, making available parent `p`, event `ev`, and target `e`

- `def Prev(code, event)`
    An async surreal.js script block event handler for `event` on previous sibling, with same vars as `On`

- `def Now(code, sel)`
    An async surreal.js script block on selector `me(sel)`

- `def AnyNow(sel, code)`
    An async surreal.js script block on selector `any(sel)`

- `def run_js(js, id, **kw)`
    Run `js` script, auto-generating `id` based on name of caller if needed, and js-escaping any `kw` params

- `def jsd(org, repo, root, path, prov, typ, ver, esm, **kwargs)`
    jsdelivr `Script` or CSS `Link` tag, or URL

- `class Fragment`
    An empty tag, used as a container

    - `def __init__(self, *c)`

- `@delegates(ft_hx, keep=True) def Titled(title, *args, **kwargs)`
    An HTML partial containing a `Title`, and `H1`, and any provided children

- `def Socials(title, site_name, description, image, url, w, h, twitter_site, creator, card)`
    OG and Twitter social card headers

- `def YouTubeEmbed(video_id, **kwargs)`
    Embed a YouTube video

- `def Favicon(light_icon, dark_icon)`
    Light and dark favicon headers
</doc><doc title="MonsterUI API List" desc="Complete API Reference for Monster UI, a component framework similar to shadcn, but for FastHTML"># monsterui Module Documentation

## monsterui.core

- `class ThemeRadii(Enum)`
    Members: none, sm, md, lg


- `class ThemeShadows`

- `class ThemeFont`

- `class Theme(Enum)`
    Selector to choose theme and get all headers needed for app.  Includes frankenui + tailwind + daisyui + highlight.js options
    Members: slate, stone, gray, neutral, red, rose, orange, green, blue, yellow, violet, zinc

    - `headers(self, mode, icons, daisy, highlightjs, katex, apex_charts, radii, shadows, font)`
        Create frankenui and tailwind cdns

    - `local_headers(self, mode, static_dir, icons, daisy, highlightjs, katex, apex_charts, radii, shadows, font)`
        Create headers using local files downloaded from CDNs


## monsterui.daisy

- `class AlertT(Enum)`
    Alert styles from DaisyUI
    Members: info, success, warning, error


- `def Alert(*c, **kwargs)`
    Alert informs users about important events.

- `class StepsT(Enum)`
    Options for Steps
    Members: vertical, horizonal


- `class StepT(Enum)`
    Step styles for LiStep
    Members: primary, secondary, accent, info, success, warning, error, neutral


- `def Steps(*li, **kwargs)`
    Creates a steps container

- `def LiStep(*c, **kwargs)`
    Creates a step list item

- `class LoadingT(Enum)`
    Members: spinner, dots, ring, ball, bars, infinity, xs, sm, md, lg


- `def Loading(cls, htmx_indicator, **kwargs)`
    Creates a loading animation component

- `class ToastHT(Enum)`
    Horizontal position for Toast
    Members: start, center, end


- `class ToastVT(Enum)`
    Vertical position for Toast
    Members: top, middle, bottom


## monsterui.foundations

> Data Structures and Utilties

- `def stringify(o)`
    Converts input types into strings that can be passed to FT components

- `class VEnum(Enum)`
    Members: 

    - `__str__(self)`
    - `__add__(self, other)`
    - `__radd__(self, other)`

## monsterui.franken

- `class TextT(Enum)`
    Text Styles from https://franken-ui.dev/docs/text
    Members: paragraph, lead, meta, gray, italic, xs, sm, lg, xl, light, normal, medium, bold, extrabold, muted, primary, secondary, success, warning, error, info, left, right, center, justify, start, end, top, middle, bottom, truncate, break_, nowrap, underline, highlight


- `class TextPresets(Enum)`
    Common Typography Presets
    Members: muted_sm, muted_lg, bold_sm, bold_lg, md_weight_sm, md_weight_muted


- `def CodeSpan(*c, **kwargs)`
    A CodeSpan with Styling

- `def CodeBlock(*c, **kwargs)`
    CodeBlock with Styling

- `def H1(*c, **kwargs)`
    H1 with styling and appropriate size

- `def H2(*c, **kwargs)`
    H2 with styling and appropriate size

- `def H3(*c, **kwargs)`
    H3 with styling and appropriate size

- `def H4(*c, **kwargs)`
    H4 with styling and appropriate size

- `def H5(*c, **kwargs)`
    H5 with styling and appropriate size

- `def H6(*c, **kwargs)`
    H6 with styling and appropriate size

- `def Subtitle(*c, **kwargs)`
    Styled muted_sm text designed to go under Headings and Titles

- `def Q(*c, **kwargs)`
    Styled quotation mark

- `def Em(*c, **kwargs)`
    Styled emphasis text

- `def Strong(*c, **kwargs)`
    Styled strong text

- `def I(*c, **kwargs)`
    Styled italic text

- `def Small(*c, **kwargs)`
    Styled small text

- `def Mark(*c, **kwargs)`
    Styled highlighted text

- `def Del(*c, **kwargs)`
    Styled deleted text

- `def Ins(*c, **kwargs)`
    Styled inserted text

- `def Sub(*c, **kwargs)`
    Styled subscript text

- `def Sup(*c, **kwargs)`
    Styled superscript text

- `def Blockquote(*c, **kwargs)`
    Blockquote with Styling

- `def Caption(*c, **kwargs)`
    Styled caption text

- `def Cite(*c, **kwargs)`
    Styled citation text

- `def Time(*c, **kwargs)`
    Styled time element

- `def Address(*c, **kwargs)`
    Styled address element

- `def Abbr(*c, **kwargs)`
    Styled abbreviation with dotted underline

- `def Dfn(*c, **kwargs)`
    Styled definition term with italic and medium weight

- `def Kbd(*c, **kwargs)`
    Styled keyboard input with subtle background

- `def Samp(*c, **kwargs)`
    Styled sample output with subtle background

- `def Var(*c, **kwargs)`
    Styled variable with italic monospace

- `def Figure(*c, **kwargs)`
    Styled figure container with card-like appearance

- `def Details(*c, **kwargs)`
    Styled details element

- `def Summary(*c, **kwargs)`
    Styled summary element

- `def Data(*c, **kwargs)`
    Styled data element

- `def Meter(*c, **kwargs)`
    Styled meter element

- `def S(*c, **kwargs)`
    Styled strikethrough text (different semantic meaning from Del)

- `def U(*c, **kwargs)`
    Styled underline (for proper names in Chinese, proper spelling etc)

- `def Output(*c, **kwargs)`
    Styled output element for form results

- `def PicSumImg(h, w, id, grayscale, blur, **kwargs)`
    Creates a placeholder image using https://picsum.photos/

- `def AccordionItem(title, *c)`
    Creates a single item for use within an Accordion component, handling title, content, and open state.

- `def Accordion(*c, **kwargs)`
    Creates a styled Accordion container using accordion component.

- `class ButtonT(Enum)`
    Options for styling Buttons
    Members: default, ghost, primary, secondary, destructive, text, link, xs, sm, lg, xl, icon


- `def Button(*c, **kwargs)`
    Button with Styling (defaults to `submit` for form submission)

- `class ContainerT(Enum)`
    Max width container sizes from https://franken-ui.dev/docs/container
    Members: xs, sm, lg, xl, expand


- `class BackgroundT(Enum)`
    Members: muted, primary, secondary, default


- `def Container(*c, **kwargs)`
    Div to be used as a container that often wraps large sections or a page of content

- `def Titled(title, *c, **kwargs)`
    Creates a standard page structure for titled page.  Main(Container(title, content))

- `class DividerT(Enum)`
    Divider Styles from https://franken-ui.dev/docs/divider
    Members: icon, sm, vertical


- `def Divider(*c, **kwargs)`
    Divider with default styling and margin

- `def DividerSplit(*c)`
    Creates a simple horizontal line divider with configurable thickness and vertical spacing

- `def Article(*c, **kwargs)`
    A styled article container for blog posts or similar content

- `def ArticleTitle(*c, **kwargs)`
    A title component for use within an Article

- `def ArticleMeta(*c, **kwargs)`
    A metadata component for use within an Article showing things like date, author etc

- `class SectionT(Enum)`
    Section styles from https://franken-ui.dev/docs/section
    Members: default, muted, primary, secondary, xs, sm, lg, xl, remove_vertical


- `def Section(*c, **kwargs)`
    Section with styling and margins

- `def Form(*c, **kwargs)`
    A Form with default spacing between form elements

- `def Fieldset(*c, **kwargs)`
    A Fieldset with default styling

- `def Legend(*c, **kwargs)`
    A Legend with default styling

- `def Input(*c, **kwargs)`
    An Input with default styling

- `def Radio(*c, **kwargs)`
    A Radio with default styling

- `def CheckboxX(*c, **kwargs)`
    A Checkbox with default styling

- `def Range(*c, **kwargs)`
    A Range with default styling

- `def TextArea(*c, **kwargs)`
    A Textarea with default styling

- `def Switch(*c, **kwargs)`
    A Switch with default styling

- `def Upload(*c, **kwargs)`
    A file upload component with default styling

- `def UploadZone(*c, **kwargs)`
    A file drop zone component with default styling

- `def FormLabel(*c, **kwargs)`
    A Label with default styling

- `class LabelT(Enum)`
    Members: primary, secondary, destructive


- `def Label(*c, **kwargs)`
    FrankenUI labels, which look like pills

- `def UkFormSection(title, description, *c)`
    A form section with a title, description and optional button

- `def GenericLabelInput(label, lbl_cls, input_cls, container, cls, id, input_fn, **kwargs)`
    `Div(Label,Input)` component with Uk styling injected appropriately. Generally you should higher level API, such as `LabelInput` which is created for you in this library

- `def LabelInput(label, lbl_cls, input_cls, cls, id, **kwargs)`
    A `FormLabel` and `Input` pair that provides default spacing and links/names them based on id

- `def LabelRadio(label, lbl_cls, input_cls, container, cls, id, **kwargs)`
    A FormLabel and Radio pair that provides default spacing and links/names them based on id

- `def LabelCheckboxX(label, lbl_cls, input_cls, container, cls, id, **kwargs)`
    A FormLabel and CheckboxX pair that provides default spacing and links/names them based on id

- `def Options(*c)`
    Helper function to wrap things into `Option`s for use in `Select`

- `def Select(*option, **kwargs)`
    Creates a select dropdown with uk styling and option for adding a search box

- `def LabelSelect(*option, **kwargs)`
    A FormLabel and Select pair that provides default spacing and links/names them based on id

- `@delegates(GenericLabelInput, but=['input_fn', 'cls']) def LabelRange(label, lbl_cls, input_cls, cls, id, value, min, max, step, label_range, **kwargs)`
    A FormLabel and Range pair that provides default spacing and links/names them based on id

- `class AT(Enum)`
    Link styles from https://franken-ui.dev/docs/link
    Members: muted, text, reset, primary, classic


- `class ListT(Enum)`
    List styles using Tailwind CSS
    Members: disc, circle, square, decimal, hyphen, bullet, divider, striped


- `def ModalContainer(*c, **kwargs)`
    Creates a modal container that components go in

- `def ModalDialog(*c, **kwargs)`
    Creates a modal dialog

- `def ModalHeader(*c, **kwargs)`
    Creates a modal header

- `def ModalBody(*c, **kwargs)`
    Creates a modal body

- `def ModalFooter(*c, **kwargs)`
    Creates a modal footer

- `def ModalTitle(*c, **kwargs)`
    Creates a modal title

- `def ModalCloseButton(*c, **kwargs)`
    Creates a button that closes a modal with js

- `def Modal(*c, **kwargs)`
    Creates a modal with the appropriate classes to put the boilerplate in the appropriate places for you

- `def Placeholder(*c, **kwargs)`
    Creates a placeholder

- `def Progress(*c, **kwargs)`
    Creates a progress bar

- `def UkIcon(icon, height, width, stroke_width, cls, **kwargs)`
    Creates an icon using lucide icons

- `def UkIconLink(icon, height, width, stroke_width, cls, button, **kwargs)`
    Creates an icon link using lucide icons

- `def DiceBearAvatar(seed_name, h, w)`
    Creates an Avatar using https://dicebear.com/

- `def Center(*c, **kwargs)`
    Centers contents both vertically and horizontally by default

- `class FlexT(Enum)`
    Flexbox modifiers using Tailwind CSS
    Members: block, inline, left, center, right, between, around, stretch, top, middle, bottom, row, row_reverse, column, column_reverse, nowrap, wrap, wrap_reverse


- `def Grid(*div, **kwargs)`
    Creates a responsive grid layout with smart defaults based on content

- `def DivFullySpaced(*c, **kwargs)`
    Creates a flex div with it's components having as much space between them as possible

- `def DivCentered(*c, **kwargs)`
    Creates a flex div with it's components centered in it

- `def DivLAligned(*c, **kwargs)`
    Creates a flex div with it's components aligned to the left

- `def DivRAligned(*c, **kwargs)`
    Creates a flex div with it's components aligned to the right

- `def DivVStacked(*c, **kwargs)`
    Creates a flex div with it's components stacked vertically

- `def DivHStacked(*c, **kwargs)`
    Creates a flex div with it's components stacked horizontally

- `class NavT(Enum)`
    Members: default, primary, secondary


- `def NavContainer(*li, **kwargs)`
    Creates a navigation container (useful for creating a sidebar navigation).  A Nav is a list (NavBar is something different)

- `def NavParentLi(*nav_container, **kwargs)`
    Creates a navigation list item with a parent nav for nesting

- `def NavDividerLi(*c, **kwargs)`
    Creates a navigation list item with a divider

- `def NavHeaderLi(*c, **kwargs)`
    Creates a navigation list item with a header

- `def NavSubtitle(*c, **kwargs)`
    Creates a navigation subtitle

- `def NavCloseLi(*c, **kwargs)`
    Creates a navigation list item with a close button

- `class ScrollspyT(Enum)`
    Members: underline, bold


- `def NavBar(*c)`
    Creates a responsive navigation bar with mobile menu support

- `def SliderContainer(*c, **kwargs)`
    Creates a slider container

- `def SliderItems(*c, **kwargs)`
    Creates a slider items container

- `def SliderNav(cls, prev_cls, next_cls, **kwargs)`
    Navigation arrows for Slider component

- `def Slider(*c, **kwargs)`
    Creates a slider with optional navigation arrows

- `def DropDownNavContainer(*li, **kwargs)`
    A Nav that is part of a DropDown

- `def TabContainer(*li, **kwargs)`
    A TabContainer where children will be different tabs

- `class CardT(Enum)`
    Card styles from UIkit
    Members: default, primary, secondary, destructive, hover


- `def CardTitle(*c, **kwargs)`
    Creates a card title

- `def CardHeader(*c, **kwargs)`
    Creates a card header

- `def CardBody(*c, **kwargs)`
    Creates a card body

- `def CardFooter(*c, **kwargs)`
    Creates a card footer

- `def CardContainer(*c, **kwargs)`
    Creates a card container

- `def Card(*c, **kwargs)`
    Creates a Card with a header, body, and footer

- `class TableT(Enum)`
    Members: divider, striped, hover, sm, lg, justify, middle, responsive


- `def Table(*c, **kwargs)`
    Creates a table

- `def TableFromLists(header_data, body_data, footer_data, header_cell_render, body_cell_render, footer_cell_render, cls, sortable, **kwargs)`
    Creates a Table from a list of header data and a list of lists of body data

- `def TableFromDicts(header_data, body_data, footer_data, header_cell_render, body_cell_render, footer_cell_render, cls, sortable, **kwargs)`
    Creates a Table from a list of header data and a list of dicts of body data

- `def apply_classes(html_str, class_map, class_map_mods)`
    Apply classes to html string

- `def render_md(md_content, class_map, class_map_mods)`
    Renders markdown using mistletoe and lxml

- `def get_franken_renderer(img_dir)`
    Create a renderer class with the specified img_dir

- `def ThemePicker(color, radii, shadows, font, mode, cls, custom_themes)`
    Theme picker component with configurable sections

- `def LightboxContainer(*lightboxitem, **kwargs)`
    Lightbox container that will hold `LightboxItems`

- `def LightboxItem(*c, **kwargs)`
    Anchor tag with appropriate structure to go inside a `LightBoxContainer`

- `def ApexChart(**kws)`
    Apex chart component
</doc></api><examples><doc title="Websockets application" desc="Very brief example of using websockets with HTMX and FastHTML">from asyncio import sleep
from fasthtml.common import *

app = FastHTML(exts='ws')
rt = app.route

def mk_inp(): return Input(id='msg')
nid = 'notifications'

@rt('/')
async def get():
    cts = Div(
        Div(id=nid),
        Form(mk_inp(), id='form', ws_send=True),
        hx_ext='ws', ws_connect='/ws')
    return Titled('Websocket Test', cts)

async def on_connect(send): await send(Div('Hello, you have connected', id=nid))
async def on_disconnect( ): print('Disconnected!')

@app.ws('/ws', conn=on_connect, disconn=on_disconnect)
async def ws(msg:str, send):
    await send(Div('Hello ' + msg, id=nid))
    await sleep(2)
    return Div('Goodbye ' + msg, id=nid), mk_inp()

serve()
</doc><doc title="Todo list application" desc="Detailed walk-thru of a complete CRUD app in FastHTML showing idiomatic use of FastHTML and HTMX patterns.">### Walkthrough of an idiomatic fasthtml app ###

# This fasthtml app includes functionality from fastcore, starlette, fastlite, and fasthtml itself.
# Run with: `python adv_app.py`
# Importing from `fasthtml.common` brings the key parts of all of these together. We recommend using a wildcard import since only necessary parts are exported by the module.
from fasthtml.common import *
from hmac import compare_digest

# We recommend using sqlite for most apps, as it is simple, fast, and scalable. `database()` creates the db if it doesn't exist.
db = database('data/utodos.db')
# Create regular classes for your database tables. There are auto-converted to fastcore flexiclasses, which are like dataclasses, but with some extra functionality.
class User: name:str; pwd:str
class Todo: id:int; title:str; done:bool; name:str; details:str; priority:int
# The `create` method creates a table in the database, if it doesn't already exist. The `pk` argument specifies the primary key for the table. If not provided, it defaults to 'id'.
users = db.create(User, pk='name')
# The `transform` argument is used to automatically update the database table, if it exists, to match the class definition. It is a simple and effective migration system for less complex needs. Use the `fastmigrate` package for more sophisticated migrations.
todos = db.create(Todo, transform=True)

# Any Starlette response class can be returned by a FastHTML route handler. In that case, FastHTML won't change it at all.
login_redir = RedirectResponse('/login', status_code=303)

# The `before` function is a *Beforeware* function. These are functions that run before a route handler is called.
def before(req, sess):
    # This sets the `auth` attribute in the request scope, and gets it from the session. The session is a Starlette session, which is a dict-like object which is cryptographically signed, so it can't be tampered with.
    # The `auth` key in the scope is automatically provided to any handler which requests it, and can not be injected by the user using query params, cookies, etc, so it should be secure to use.
    auth = req.scope['auth'] = sess.get('auth', None)
    if not auth: return login_redir
    # `xtra` adds a filter to queries and DDL statements, to ensure that the user can only see/edit their own todos.
    todos.xtra(name=auth)

# Beforeware objects require the function itself, and optionally a list of regexes to skip.
bware = Beforeware(before, skip=[r'/favicon\.ico', r'/static/.*', r'.*\.css', '/login', '/send_login'])

markdown_js = """
import { marked } from "https://cdn.jsdelivr.net/npm/marked/lib/marked.esm.js";
proc_htmx('.markdown', e => e.innerHTML = marked.parse(e.textContent));
"""

# The `FastHTML` class is a subclass of `Starlette`, so you can use any parameters that `Starlette` accepts. In addition, you can add your Beforeware here, and any headers you want included in HTML responses.
def _not_found(req, exc): return Titled('Oh no!', Div('We could not find that page :('))
app = FastHTML(before=bware,
               # These are the same as Starlette exception_handlers, except they also support `FT` results
               exception_handlers={404: _not_found},
               # PicoCSS is a simple CSS system for getting started; for more complex styling try MonsterUI (which wraps uikit and Tailwind)
               hdrs=(picolink, # PicoCSS headers
                     # Look at fasthtml/js.py to see how to add Javascript libraries to FastHTML, like this one.
                     SortableJS('.sortable'),
                     # MarkdownJS is actually provided as part of FastHTML, but we've included the js code here so that you can see how it works.
                     Script(markdown_js, type='module'))
                )
# We add `rt` as a shortcut for `app.route`, which is what we'll use to decorate our route handlers.
rt = app.route

# FastHTML uses Starlette's path syntax, and adds a `static` type which matches standard static file extensions. You can define your own regex path specifiers -- for instance this is how `static` is defined in FastHTML `reg_re_param("static", "ico|gif|jpg|jpeg|webm|css|js|woff|png|svg|mp4|webp|ttf|otf|eot|woff2|txt|xml|html")`
# Provide param to `rt` to use full Starlette route syntax.
@rt("/{fname:path}.{ext:static}", methods=['GET'])
def static_handler(fname:str, ext:str): return FileResponse(f'{fname}.{ext}')

# This function handles GET and POST requests to the `/login` path, because the name of the function automatically becomes the path for the route handler, and GET/POST are available by default. We recommend generally sticking to just these two HTTP verbs.
@rt
def login():
    # This creates a form with two input fields, and a submit button. `Input`, `Form`, etc are `FT` (fasttag) objects. FastHTML composes them from trees and auto-converts them to HTML when needed.
    # You can also use plain HTML strings in handlers and headers, which will be auto-escaped, unless you use `Safe(...string...)`. If you want other custom tags (e.g. `MyTag`), they can be auto-generated by e.g:
    #   `from fasthtml.components import MyTag`.
    # fasttag objects are callable. Calling them adds children and attributes to the tag. Therefore you can use them like this:
    frm = Form(action=send_login, method='post')(
        # Tags with a `name` attr will have `name` auto-set to the same as `id` if not provided
        Input(id='name', placeholder='Name'),
        Input(id='pwd', type='password', placeholder='Password'),
        Button('login'))
    # If a user visits the URL directly, FastHTML auto-generates a full HTML page. However, if the URL is accessed by HTMX, then one HTML partial is created for each element of the tuple.
    # To avoid this auto-generation of a full page, return a `HTML` object, or a Starlette `Response`.
    # `Titled` returns a tuple of a `Title` with the first arg and a `Container` with the rest.
    # A handler can return either a single `FT` object or string, or a tuple of them.
    # In the case of a tuple, the stringified objects are concatenated and returned to the browser. The `Title` tag has a special purpose: it sets the title of the page (this is HTMX's built in behavior for title HTML partials).
    return Titled("Login", frm)

# Handlers are passed whatever information they "request" in the URL, as keyword arguments.
# This handler is called when a POST request is made to the `/login` path. The `login` argument is an instance of the `Login` class, which has been auto-instantiated from the form data.
# There are a number of special parameter names, which will be passed useful information about the request: `session`: the Starlette session; `request`: the Starlette request; `auth`: the value of `scope['auth']`, `htmx`: the HTMX headers, if any; `app`: the FastHTML app object.
# You can also pass any string prefix of `request` or `session`.
@rt
def send_login(name:str, pwd:str, sess):
    if not name or not pwd: return login_redir
    # Indexing into a table queries by primary key, which is `name` here.
    try: u = users[name]
    # If the primary key does not exist, the method raises a `NotFoundError`. Here we use this to just generate a user -- in practice you'd probably to redirect to a signup page.
    # Note that `insert` (and all similar db methods) returns the row object, so we can use it to get the new user.
    except NotFoundError: u = users.insert(name=name, pwd=pwd)
    if not compare_digest(u.pwd.encode("utf-8"), pwd.encode("utf-8")): return login_redir
    # Because the session is signed, we can securely add information to it. It's stored in the browser cookies. If you don't pass a secret signing key to `FastHTML`, it will auto-generate one and store it in a file `./sesskey`.
    sess['auth'] = u.name
    return RedirectResponse('/', status_code=303)

@rt
def logout(sess):
    del sess['auth']
    return login_redir

# Refactoring components in FastHTML is as simple as creating Python functions. The `clr_details` function creates a Div with specific HTMX attributes.
# `hx_swap_oob='innerHTML'` tells HTMX to swap the inner HTML of the target element out-of-band, meaning it will update this element regardless of where the HTMX request originated from. This returned div is empty, so it will clear the details view.
def clr_details(): return Div(hx_swap_oob='innerHTML', id='current-todo')

# Dataclasses, dicts, namedtuples, TypedDicts, and custom classes are automatically instantiated from form data.
# In this case, the `Todo` class is a flexiblass (a subclass of dataclass), so the handler will be passed all the field names of it.
@rt
def update(todo: Todo):
    # The updated todo is returned. By returning the updated todo, we can update the list directly. Because we return a tuple with `clr_details()`, the details view is also cleared.
    # Later on, the `__ft__` method of the `Todo` class will be called to convert it to a fasttag.
    return todos.update(todo), clr_details()

@rt
def edit(id:int):
    # `target_id` specifies which element will be updated with the server's response (it's a shortcut for hx_target=f"#{...}").
    # CheckboxX add an extra hidden field with the same name as the checkbox, so that it can be submitted as part of the form data. This is useful for boolean fields, where you want to know if the field was checked or not.
    res = Form(hx_post=update, target_id=f'todo-{id}', id="edit")(
        Group(Input(id="title"), Button("Save")),
        Hidden(id="id"), CheckboxX(id="done", label='Done'),
        Textarea(id="details", name="details", rows=10))
    # `fill_form` populates the form with existing todo data, and returns the result. Indexing into a table (`todos`) queries by primary key, which is `id` here. It also includes `xtra`, so this will only return the id if it belongs to the current user.
    return fill_form(res, todos[id])

@rt
def rm(id:int):
    # `delete` removes the item with the given primary key.
    todos.delete(id)
    # Returning `clr_details()` ensures the details view is cleared after deletion, leveraging HTMX's out-of-band swap feature.
    # Note that we are not returning *any* FT component that doesn't have an "OOB" swap, so the target element inner HTML is simply deleted.
    return clr_details()

@rt
def show(id:int):
    todo = todos[id]
    # `hx_swap` determines how the update should occur. We use "outerHTML" to replace the entire todo `Li` element.
    # `rm.to(id=todo.id)` is a shortcut for `f'/rm?id={todo.id}'`. All routes have this `to` method.
    btn = Button('delete', hx_post=rm.to(id=todo.id),
                 hx_target=f'#todo-{todo.id}', hx_swap="outerHTML")
    # The "markdown" class is used here because that's the CSS selector we used in the JS earlier. This will trigger the JS to parse the markdown.
    # Because `class` is a reserved keyword in Python, we use `cls` instead, which FastHTML auto-converts.
    return Div(H2(todo.title), Div(todo.details, cls="markdown"), btn)

# `fastcore.patch` adds a method to an existing class.
# The `__ft__` method is a special method that FastHTML uses to convert the object into an `FT` object, so that it can be composed into an FT tree, and later rendered into HTML.
@patch
def __ft__(self:Todo):
    # Some FastHTML tags have an 'X' suffix, which means they're "extended" in some way. For instance, here `AX` is an extended `A` tag, which takes 3 positional arguments: `(text, hx_get, target_id)`.
    # All underscores in FT attrs are replaced with hyphens, so this will create an `hx-get` attr, which HTMX uses to trigger a GET request.
    # Generally, most of your route handlers in practice (as in this demo app) are likely to be HTMX handlers.
    ashow = AX(self.title, show.to(id=self.id), 'current-todo')
    aedit = AX('edit',     edit.to(id=self.id), 'current-todo')
    dt = '✅ ' if self.done else ''
    # FastHTML provides some shortcuts. For instance, `Hidden` is defined as simply: `return Input(type="hidden", value=value, **kwargs)`
    cts = (dt, ashow, ' | ', aedit, Hidden(id="id", value=self.id), Hidden(id="priority", value="0"))
    # Any FT object can take a list of children as positional args, and a dict of attrs as keyword args.
    return Li(*cts, id=f'todo-{self.id}')

@rt
def create(todo:Todo):
    # `hx_swap_oob='true'` tells HTMX to perform an out-of-band swap, updating this element wherever it appears. This is used to clear the input field after adding the new todo.
    new_inp =  Input(id="new-title", name="title", placeholder="New Todo", hx_swap_oob='true')
    # `insert` returns the inserted todo, which is appended to the list start, because we used `hx_swap='afterbegin'` when creating the form.
    return todos.insert(todo), new_inp

# Because the todo list form created earlier included hidden inputs with the todo IDs, they are included in the form data. By using a parameter called (e.g) "id", FastHTML will try to find something suitable in the request with this name. In order, it searches as follows: path; query; cookies; headers; session keys; form data.
# FastHTML will use your parameter's type annotation to try to cast the value to the requested type. In the case of form data, there can be multiple values with the same key. So in this case, the parameter is a list of ints.
@rt
def reorder(id:list[int]):
    # Normally running a query in a loop like this would be really slow. But sqlite is at least as fast as a file system, so this pattern is actually idiomatic and efficient.
    for i,id_ in enumerate(id): todos.update({'priority':i}, id_)
    # HTMX by default replaces the inner HTML of the calling element, which in this case is the todo list form. Therefore, we return the list of todos, now in the correct order, which will be auto-converted to FT for us.
    # In this case, it's not strictly necessary, because sortable.js has already reorder the DOM elements. However, by returning the updated data, we can be assured that there aren't sync issues between the DOM and the server.
    return tuple(todos(order_by='priority'))

# This is the handler for the main todo list application. By including the `auth` parameter, it gets passed the current username, for displaying in the title. `index()` is a special name for the main route handler, and is called when the root path `/` is accessed.
@rt
def index(auth):
    title = f"{auth}'s Todo list"
    top = Grid(H1(title), Div(A('logout', href=logout), style='text-align: right'))
    new_inp = Input(id="new-title", name="title", placeholder="New Todo")
    add = Form(Group(new_inp, Button("Add")),
               hx_post=create, target_id='todo-list', hx_swap="afterbegin")
    # Treating a table as a callable (i.e with `todos(...)` here) queries the table. Because we called `xtra` in our Beforeware, this queries the todos for the current user only.
    # We can include the todo objects directly as children of the `Form`, because the `Todo` class has `__ft__` defined. This is automatically called by FastHTML to convert the `Todo` objects into `FT` objects when needed.
    # The reason we put the todo list inside a form is so that we can use the 'sortable' js library to reorder them. That library calls the js `end` event when dragging is complete, so our trigger here causes our `/reorder` handler to be called.
    frm = Form(*todos(order_by='priority'),
               id='todo-list', cls='sortable', hx_post=reorder, hx_trigger="end")
    # We create an empty 'current-todo' Div at the bottom of our page, as a target for the details and editing views.
    card = Card(P('Drag/drop todos to reorder them'),
                Ul(frm),
                header=add, footer=Div(id='current-todo'))
    # PicoCSS uses `<Main class='container'>` page content; `Container` is a tiny function that generates that.
    return Title(title), Container(top, card)

# You do not need `if __name__ == '__main__':` in FastHTML apps, because `serve()` handles this automatically. By default it reloads the app when the source code changes.
serve()</doc></examples></project>

Overall summary upto this point.
---

**Learner Context**: New to web programming, comfortable with Python, prefers learning through discussion and exploring concepts.

**Chapters Covered**: "Hypermedia: A Reintroduction" and "Components of a Hypermedia System" from "Hypermedia Systems"

**Key Concepts Discussed**:

1. **Hypermedia Definition**: Media with non-linear branching (e.g., hyperlinks). A **hypermedia control** encodes interaction information directly and completely within itself — self-describing, no external knowledge needed. Think about the *whole system*: HTML + HTTP + browser + server working together.

2. **HTML's Two Hypermedia Controls**: Anchor tags (navigation, GET) and form tags (data submission, GET/POST). Forms enabled the web to become an *application* platform, not just documents. GET appends data to URL (bookmarkable); POST sends data in body. POST-Redirect-GET pattern prevents duplicate submissions.

3. **SPAs vs Hypermedia**: SPAs exchange JSON data, require client-side code with intimate knowledge of API structure (**tight coupling**). Hypermedia exchanges HTML with embedded controls — server can change without breaking client (**loose coupling**). SPAs shift code/complexity to the browser.

4. **Why SPAs Won**: HTML froze as hypermedia in the mid-90s; user experience demands grew; JavaScript filled the gap. Won on *user experience*, not architectural superiority.

5. **Hypermedia-Driven Applications (HDAs)**: Apps using hypermedia exchanges as primary server communication. Libraries like **htmx** extend HTML's capabilities with declarative attributes (`hx-get`, `hx-target`). **Important**: browsers don't natively understand these attributes — the htmx library scans for them, attaches behavior, handles requests/swaps; the browser just renders the resulting HTML.

6. **Transitional vs Hypermedia-First**: Rich Harris's "transitional" approach = SPA as home base with MPA sprinkled in. The book's approach = **hypermedia as default**, SPA only when genuinely needed. The mindset is flipped.

7. **When to Use What**: Plain HTML → many apps. HTML + htmx → most interactivity needs. SPA → only when required (e.g., spreadsheets with cascading updates on every keystroke). **Can mix**: hypermedia for simple parts, SPA for complex features — save your complexity budget.

8. **Browser Benefits "For Free"**: Back button, deep linking, caching — all work naturally with hypermedia; must be manually reimplemented in SPAs.

9. **Four Components of a Hypermedia System**: Hypermedia (HTML), network protocol (HTTP), hypermedia server (responds to requests), hypermedia client (browser interprets responses). All four must work together.

10. **URLs**: Uniform Resource Locators uniquely identify resources. Components: scheme (protocol), host (domain), port, path, query, fragment. Relative URLs inherit from current document.

11. **HTTP Basics**: Simple request/response protocol. Request headers (like `Accept`) tell server preferences; response headers provide metadata. Content negotiation allows same URL to return different representations.

12. **HTTP Methods**: GET (read), POST (create/catch-all), PUT (replace), PATCH (partial update), DELETE (remove). Map roughly to CRUD. HTML only supports GET and POST natively — a limitation htmx addresses. Methods are conventions, not enforced constraints.

13. **HTTP Response Codes**: 1xx informational, 2xx success, 3xx redirect, 4xx client error, 5xx server error. Subtle differences matter (302 vs 303). Using codes properly goes "with the grain" of the web.

14. **Caching**: `Cache-Control` header indicates cacheability. `Vary` header tells caches which request headers affect the response (important for htmx's `HX-Request` header).

15. **Hypermedia Servers**: Any language works — hypermedia frees you from JavaScript lock-in. "HOWL" = Hypermedia On Whatever you'd Like.

16. **Hypermedia Clients**: Must generically interpret hypermedia controls. Browsers don't know what a "contact" is — they just render HTML. JSON API clients expect fixed formats, which is why HATEOAS hasn't caught on there.

17. **REST (Original Meaning)**: Fielding's dissertation described the pre-API web. Most "REST APIs" today aren't RESTful by his definition.

18. **REST Constraints**:
    - *Client-Server*: Obvious for web apps
    - *Statelessness*: Each request self-contained. Sessions (cookies) technically violate this — pragmatic trade-off
    - *Caching*: Responses indicate cacheability
    - *Uniform Interface*: The core innovation (see below)
    - *Layered System*: Intermediaries (CDNs, proxies) allowed
    - *Code-On-Demand*: Scripting is optional — should augment, not replace hypermedia

19. **Uniform Interface Sub-constraints**:
    - *Identification of resources*: URLs uniquely identify resources
    - *Manipulation through representations*: You exchange representations, not resources directly
    - *Self-descriptive messages*: All info needed is in the response itself
    - *HATEOAS*: Hypermedia As The Engine of Application State — available actions encoded in HTML, not in client code

20. **HATEOAS & API Churn**: Hypermedia APIs don't need versioning. Server changes, client automatically gets new controls. JSON APIs require documentation exchange and coordinated updates.

21. **Core Trade-off**: Hypermedia is less efficient (more bytes) but vastly more flexible — clients need zero domain knowledge, just render what arrives.

In [ ]:
from dialoghelper import *

In [ ]:
# tool_info()

Tools available from `dialoghelper`:

- &`curr_dialog`: Get the current dialog info.
- &`msg_idx`: Get absolute index of message in dialog.
- &`add_html`: Send HTML to the browser to be swapped into the DOM using hx-swap-oob.
- &`find_msg_id`: Get the current message id.
- &`find_msgs`: Find messages in current specific dialog that contain the given information.
  - (solveit can often get this id directly from its context, and will not need to use this if the required information is already available to it.)
- &`read_msg`: Get the message indexed in the current dialog.
  - To get the exact message use `n=0` and `relative=True` together with `msgid`.
  - To get a relative message use `n` (relative position index).
  - To get the nth message use `n` with `relative=False`, e.g `n=0` first message, `n=-1` last message.
- &`del_msg`: Delete a message from the dialog.
- &`add_msg`: Add/update a message to the queue to show after code execution completes.
- &`update_msg`: Update an existing message.
- &`url2note`: Read URL as markdown, and add a note below current message with the result
- &`msg_insert_line`: Insert text at a specific location in a message.
- &`msg_str_replace`: Find and replace text in a message.
- &`msg_strs_replace`: Find and replace multiple strings in a message.
- &`msg_replace_lines`: Replace a range of lines in a message with new content.
  - Always first use `read_msg( msgid=msgid, n=0, relative=True, nums=True)` to view the content with line numbers.

In [ ]:
# content_url = "https://hypermedia.systems/a-web-1-0-application/"
# content = url2note(content_url)

[Previous: Components Of A Hypermedia System](/components-of-a-hypermedia-system/)

[Next: Hypermedia-Driven Web Applications With Htmx](/part/htmx/)



# A Web 1.0 Application

Contents

  * [Picking A “Web Stack”](/a-web-1-0-application/#_picking_a_web_stack)
  * [Python](/a-web-1-0-application/#_python)
  * [Introducing Flask: Our First Route](/a-web-1-0-application/#_introducing_flask_our_first_route)
  * [Contact.app Functionality](/a-web-1-0-application/#_contact_app_functionality)
    * [Showing A Searchable List Of Contacts](/a-web-1-0-application/#_showing_a_searchable_list_of_contacts)
      * [The list & search templates](/a-web-1-0-application/#_the_list_search_templates)
    * [Adding A New Contact](/a-web-1-0-application/#_adding_a_new_contact)
      * [Handling the post to /contacts/new](/a-web-1-0-application/#_handling_the_post_to_contactsnew)
    * [Viewing The Details Of A Contact](/a-web-1-0-application/#_viewing_the_details_of_a_contact)
    * [The Contact Detail Template](/a-web-1-0-application/#_the_contact_detail_template)
    * [Editing And Deleting A Contact](/a-web-1-0-application/#_editing_and_deleting_a_contact)
      * [Handling the post to /contacts/<contact_id>/edit](/a-web-1-0-application/#_handling_the_post_to_contactscontact_id)
    * [Deleting A Contact](/a-web-1-0-application/#_deleting_a_contact)
    * [Contact.app…​ Implemented!](/a-web-1-0-application/#_contact_app_implemented)
  * [HTML Notes: Framework Soup](/a-web-1-0-application/#html-note-title)



To start our journey into Hypermedia-Driven Applications, we are going to create a simple contact management web application called Contact.app. We will start with a basic, “Web 1.0-style” Multi-Page Application (MPA), in the grand CRUD (Create, Read, Update, Delete) tradition. It will not be the best contact management application in the world, but it will be simple and it will do its job.

This application will also be easy to incrementally improve in the coming chapters by utilizing the hypermedia-oriented library htmx.

By the time we are finished building and enhancing the application, over the next few chapters, it will have some very slick features that most developers today would assume requires the use of a SPA JavaScript framework.



## Picking A “Web Stack”

In order to demonstrate how web 1.0 applications work, we need to pick a server-side language and a library for handling HTTP requests. Colloquially, this is called our “Server-Side” or “Web” stack, and there are literally hundreds of options to choose from, many with passionate followings. You probably have a web framework that you prefer and, while we wish we could write this book for every possible stack out there, in the interest of simplicity (and sanity) we can only pick one.

For this book we are going to use the following stack:

  * [Python](https://www.python.org/) as our programming language.

  * [Flask](https://palletsprojects.com/p/flask/) as our web framework, allowing us to connect HTTP requests to Python logic.

  * [Jinja2](https://palletsprojects.com/p/jinja/) for our server-side templating language, allowing us to render HTML responses using a familiar and intuitive syntax.




Why this particular stack?

Python is the most popular programming language in the world, as of this writing, according to the [TIOBE index](https://www.tiobe.com/tiobe-index/), a respected measure of programming language popularity. More importantly, Python is easy to read even if you aren’t familiar with it.

We chose the Flask web framework because it is simple and does not impose a lot of structure on top of the basics of HTTP request handling.

This bare-bones approach is a good match for our needs: in other cases you might consider a more full-featured Python framework, such as [Django](https://www.djangoproject.com/), which supplies much more functionality out of the box than Flask does.

By using Flask for our book, we will be able to keep our code focused on _hypermedia exchanges_.

We picked Jinja2 templates because they are the default templating language for Flask. They are simple enough and similar enough to most other server-side templating languages that most people who are familiar with any server-side (or client-side) templating library should be able to understand them quickly and easily.

Even if this combination of technologies isn’t your preferred stack, please, keep reading: you will learn quite a bit from the patterns we introduce in the coming chapters and it shouldn’t be hard to map them into your preferred language and frameworks.

With this stack we will be rendering HTML _on the server-side_ to return to clients, rather than producing JSON. This is the traditional approach to building web applications. However, with the rise of SPAs, this approach is not as widely used a technique as it once was. Today, as people are rediscovering this style of web applications, the term “Server-Side Rendering” or SSR is emerging as the way that people talk about it. This contrasts with “Client-Side Rendering”, that is, rendering templates in the browser with data retrieved in JSON form from the server, as is common in SPA libraries.

In Contact.app we will intentionally keep things as simple as possible to maximize the teaching value of our code: it won’t be perfectly factored code, but it will be easy to follow for readers, even if they have little Python experience, and it should be easy to translate both the application and the techniques demonstrated into your preferred programming environment.



This note talks about what we will be using to get an understanding of the concepts of hypermedia applications. Anything worth dwelling or paying attention to here?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

A couple of things worth noting:

**The "HOWL" principle in action** — Remember from our earlier discussion that hypermedia frees you from JavaScript lock-in? This is them putting that into practice. They could have picked Ruby, Go, Java, anything. They chose Python for *readability*, not because it's required.

**Server-Side Rendering (SSR) terminology** — This is interesting historically. They mention that "SSR" is a term that's *re-emerging*. It used to just be called... building a website! The fact that we needed a new term shows how much the SPA mindset had taken over. Now rendering HTML on the server is treated as a "technique" rather than the default.

**Flask's simplicity is deliberate** — They picked a minimal framework specifically so you can see the *hypermedia exchanges* without a lot of framework magic obscuring things. Django would do more for you automatically, but you'd learn less about what's actually happening.

**The SSR vs CSR distinction** — This is the crux:
- SSR: Server builds complete HTML, sends it down
- CSR (Client-Side Rendering): Server sends JSON + a JavaScript app that builds HTML in the browser

With hypermedia, the server is always in charge of generating the UI. The browser just renders what arrives.

Does the SSR/CSR distinction feel clear, or would it help to talk through what happens step-by-step in each approach?

Makes sense. Thanks

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You're welcome! I'm ready to help whenever you need. Just let me know what you'd like to work on or learn about.

## Python

Since this book is for learning how to use hypermedia effectively, we’ll just briefly introduce the various technologies we use _around_ that hypermedia. This has some obvious drawbacks: if you aren’t comfortable with Python, for example, some example Python code in the book may be a bit confusing or mysterious at first.

If you feel like you need a quick introduction to the language before diving into the code, we recommend the following books and websites:

  * [Python Crash Course](https://nostarch.com/python-crash-course-3rd-edition) from No Starch Press

  * [Learn Python The Hard Way](https://learnpythonthehardway.org/python3/) by Zed Shaw

  * [Python For Everybody](https://www.py4e.com/) by Dr. Charles R. Severance




We think most web developers, even developers who are unfamiliar with Python, should be able to follow along with our examples. Most of the authors of this book hadn’t written much Python before writing it, and we got the hang of it pretty quickly.



## Introducing Flask: Our First Route

Flask is a simple but flexible web framework for Python. We’ll ease into it by touching on its core elements.

A Flask application consists of a series of _routes_ tied to functions that execute when an HTTP request to a given path is made. It uses a Python feature called “decorators” to declare the route that will be handled, which is then followed by a function to handle requests to that route. We’ll use the term “handler” to refer to the functions associated with a route.

Let’s create our first route definition, a simple “Hello World” route. In the following Python code you will see the `@app` symbol. This is the flask decorator that allows us to set up our routes. Don’t worry too much about how decorators work in Python, just know that this feature allows us to map a given _path_ to a particular function (i.e., handler). The Flask application, when started, will take HTTP requests and look up the matching handler and invoke it.
    
    
    @app.route("/") <1>
    def index(): <2>
        return "Hello World!" <3>

A simple “Hello World” route

  1. Establishes we are mapping the `/` path as a route.

  2. The next method is the handler for that route.

  3. Returns the string “Hello World!” to the client.




The `route()` method on the Flask decorator takes an argument: the path you wish the route to handle. Here we pass in the root or `/` path, as a string, to handle requests to the root path.

This route declaration is then followed by a simple function definition, `index()`. In Python, decorators invoked in this manner apply to the function immediately following them. Therefore, this function becomes the “handler” for that route, and will be executed when an HTTP request to the given path is made.

Note that the name of the function doesn’t matter, we can call it whatever we’d like so long as it is unique. In this case we chose `index()` because that fits with the route we are handling: the root “index” of the web application.

So we have the `index()` function immediately following our route definition for the root, and this will become the handler for the root URL in our web application.

The handler in this case is dead simple, it just returns a string, “Hello World!”, to the client. This isn’t hypermedia yet, but as we can see in [fig-helloworld], a browser will render it just fine.

![](https://hypermedia.systems/images/figure_2-1_hello_world.png#ai)

Hello World!

Great, there’s our first step into Flask, showing the core technique we are going to use to respond to HTTP requests: routes mapped to handlers.

For Contact.app, rather than rendering “Hello World!” at the root path, we are going to do something a little fancy: we are going to redirect to another path, the `/contacts` path. Redirects are a feature of HTTP that allow you to redirect a client to another location with an HTTP response.

We are going to display a list of contacts as our root page, and, arguably, redirecting to the `/contacts` path to display this information is a bit more consistent with the notion of resources with REST. This is a judgement call on our part, and not something we feel is too important, but it makes sense in terms of routes we will set up later in the application.

To change our “Hello World” route to a redirect, we only need to change one line of code:
    
    
    @app.route("/")
    def index():
        return redirect("/contacts") <1>

Changing “Hello World” to a redirect

  1. Update to a call to `redirect()`




Now the `index()` function returns the result of the Flask-supplied `redirect()` function with the path we’ve supplied. In this case the path is `/contacts`, passed in as a string argument. Now, if you navigate to the root path, `/`, our Flask application will forward you on to the `/contacts` path.



Here we are seeing a bit of flask internals. Crux of the matter is creating routes which can be called with HTTP requests. In one we simply return a string to be rendered by the browser and another we redirect it to a different route. The route is not setup yet but assumption is that it exists and has some content. Did I get that right? Anything else worth pausing or dwelling?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

You've got it exactly right. Routes map paths to handler functions, and those handlers can return content directly or redirect elsewhere.

A few things worth noting:

**Decorators are just "magic annotations"** — Don't get hung up on how `@app.route("/")` works internally. Just think of it as saying "when someone requests this path, run the function below." That's the mental model you need.

**The function name is arbitrary** — They named it `index()` because it handles the root path, but you could call it `banana()` and it would work the same. The decorator is what connects the path to the function, not the name.

**Redirects are HTTP-level** — When you call `redirect("/contacts")`, Flask sends back a response with a 3xx status code (probably 302 or 303) and a `Location` header pointing to `/contacts`. The *browser* then automatically makes a new request to that location. This is the HTTP protocol doing its job — the browser knows how to follow redirects without any special client-side code.

**The REST-ish design choice** — They mention redirecting to `/contacts` is "more consistent with REST." The idea is that the *resource* is the collection of contacts, so it should have its own URL. The root `/` just points you there. This is a style choice, not a requirement.

Does the redirect mechanism make sense, or would you like to trace through what HTTP requests/responses actually happen when you visit `/`?

Does that mean `redirect("/contacts")` and `redirect("contacts")` both would have worked and authors chose the first?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good question — they're actually *not* equivalent!

- `/contacts` (with leading slash) = **absolute path** — always goes to `http://yoursite.com/contacts` regardless of where you currently are
- `contacts` (no leading slash) = **relative path** — appends to the current path

So if you were somehow at `/admin/dashboard` and redirected to `"contacts"` (relative), you'd end up at `/admin/contacts`. But `"/contacts"` (absolute) always takes you to `/contacts` from the root.

Since this handler is for `/` (the root), both would happen to work the same in this specific case. But using the absolute path is clearer and safer — it says exactly where you're going, no ambiguity.

Does that distinction feel familiar from how you've seen URLs work in HTML (like in `href` attributes)?

When the `redirect` is called; is there a get request to the path or is it just a function call?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great question — it's neither, actually! Here's what happens:

1. `redirect("/contacts")` just *builds an HTTP response* with a 3xx status code and a `Location: /contacts` header
2. Flask sends that response back to the browser
3. The *browser* sees "oh, a redirect" and automatically makes a **new GET request** to `/contacts`

So the server doesn't call anything or make any request. It just says "go look over there" and the browser obeys by making a fresh GET request.

This is why redirects show up as two separate entries if you watch your browser's network tab — the original request, then the follow-up GET.

Want to see what that response actually looks like at the HTTP level?

Ah! I see. It also makes sense to me now how Post/Redirect/Get solves the error that would exist during refresh of a post call

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Exactly! You've connected the dots perfectly.

Without PRG:
1. POST to `/submit-order`
2. Server processes, returns "Order confirmed" page
3. User hits refresh → browser tries to POST again → duplicate order!

With PRG:
1. POST to `/submit-order`
2. Server processes, returns redirect to `/order-confirmation`
3. Browser makes GET to `/order-confirmation`
4. User hits refresh → browser just GETs `/order-confirmation` again → harmless

The browser's history now points to the GET, not the POST. That's the trick.

It does. I am just not sure what the author means by `... and, arguably, redirecting to the /contacts path to display this information is a bit more consistent with the notion of resources with REST. ...` 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

This connects back to what we discussed earlier about REST and URLs identifying resources.

The idea is: in REST, each URL should identify a *thing* (a resource). So:

- `/contacts` = the collection of contacts (a resource)
- `/contacts/42` = a specific contact with ID 42 (a resource)

The root path `/` doesn't really identify a specific resource — it's just "the front door" of your app. So rather than *duplicating* the contacts list at both `/` and `/contacts`, they chose to have `/` simply point you to the "real" location of that resource.

It's a minor design preference. You could totally render the contacts list directly at `/` and skip the redirect. The app would work fine. The authors are just being tidy about "one resource, one URL."

Does that clarify the REST angle, or does the concept of URLs-as-resource-identifiers need more unpacking?

So the authors are being clean by note creating a contacts page at `/` but using `/contacts` even though they could have used it for ease 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Exactly. They *could* have just rendered the contacts list at `/` and been done with it — simpler, fewer HTTP round-trips. 

But by using the redirect, they get:
- A cleaner URL structure where `/contacts` is *the* canonical home for contacts
- Consistency if they later add other top-level resources (`/settings`, `/reports`, etc.)
- The root `/` becomes a "dispatcher" rather than owning any content itself

It's a small upfront cost (one extra redirect) for a tidier architecture. Whether that tradeoff is worth it depends on the app — for a tiny app, maybe overkill; for something that might grow, it sets good habits.

You've got the idea. Ready to move on to the actual Contact.app functionality?

Hmm. In this case the base url takes to the contact page. And there can also be a `base url/contacts` page which will return same output without redirect, correct?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Correct! You could set up two routes that both render the same contacts page:

```python
@app.route("/")
def index():
    return render_contacts()  # directly show contacts

@app.route("/contacts")
def contacts():
    return render_contacts()  # same content
```

Both would work and display the same thing. But now you have the same content at two different URLs, which is a bit "unclean" from a REST perspective — the contacts resource doesn't have a single canonical address.

The redirect approach says: "There's only *one* place where contacts live (`/contacts`), and `/` just sends you there." 

It's a style choice, not a technical requirement.

## Contact.app Functionality

Now that we have some understanding of how to define routes, let’s get down to specifying and then implementing our web application.

What will Contact.app do?

Initially, it will allow users to:

  * View a list of contacts, including first name, last name, phone and email address

  * Search the contacts

  * Add a new contact

  * View the details of a contact

  * Edit the details of a contact

  * Delete a contact




So, as you can see, Contact.app is a CRUD application, the sort of application that is perfect for an old-school web 1.0 approach.

Note that the source code of Contact.app is available on [GitHub](https://github.com/bigskysoftware/contact-app).



### Showing A Searchable List Of Contacts

Let’s add our first real bit of functionality: the ability to show all the contacts in our app in a list (really, in a table).

This functionality is going to be found at the `/contacts` path, which is the path our previous route is redirecting to.

We will use Flask to route the `/contacts` path to a handler function, `contacts()`. This function will do one of two things:

  * If there is a search term found in the request, it will filter down to only contacts matching that term

  * If not, it will simply list all contacts




This is a common approach in web 1.0 style applications: the same URL that displays all instances of some resource also serves as the search results page for those resources. Taking this approach makes it easy to reuse the list display that is common to both types of request.

Here is what the code looks like for this handler:
    
    
    @app.route("/contacts")
    def contacts():
        search = request.args.get("q") <1>
        if search is not None:
            contacts_set = Contact.search(search) <2>
        else:
            contacts_set = Contact.all() <3>
        return render_template("index.html", contacts=contacts_set)

A handler for server-side search

  1. Look for the query parameter named `q`, which stands for “query.”

  2. If the parameter exists, call the `Contact.search()` function with it.

  3. If not, call the `Contact.all()` function.

  4. Pass the result to the `index.html` template to render to the client.




We see the same sort of routing code we saw in our first example, but we have a more elaborate handler function. First, we check to see if a search query parameter named `q` is part of the request.

Query Strings
    

A “query string” is part of the URL specification. Here is an example URL with a query string in it: `https://example.com/contacts?q=joe`. The query string is everything after the `?`, and has a name-value pair format. In this URL, the query parameter `q` is set to the string value `joe`. In plain HTML, a query string can be included in a request either by being hardcoded in an anchor tag or, more dynamically, by using a form tag with a `GET` request.

To return to our Flask route, if a query parameter named `q` is found, we call out to the `search()` method on a `Contact` model object to do the actual contact search and return all the matching contacts.

If the query parameter is _not_ found, we simply get all contacts by invoking the `all()` method on the `Contact` object.

Finally, we render a template, `index.html` that displays the given contacts, passing in the results of whichever of these two functions we end up calling.

**A Note On The Contact Class**

The `Contact` Python class we’re using is the “domain model” or just “model” class for our application, providing the “business logic” around the management of Contacts.

It could be working with a database (it isn’t) or a simple flat file (it is), but we’re going to skip over the internal details of the model. Think of it as a “normal” domain model class, with methods on it that act in a “normal” manner.

We will treat `Contact` as a _resource_ , and focus on how to effectively provide hypermedia representations of that resource to clients.



Can we spend some time on this line `A “query string” is part of the URL specification. Here is an example URL with a query string in it: https://example.com/contacts?q=joe. The query string is everything after the ?, and has a name-value pair format. In this URL, the query parameter q is set to the string value joe. In plain HTML, a query string can be included in a request either by being hardcoded in an anchor tag or, more dynamically, by using a form tag with a GET request.`.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠

Sure! Let's break this down piece by piece.

**The URL structure:**
```
https://example.com/contacts?q=joe
```

- `https://example.com/contacts` — the path (what resource you're asking for)
- `?` — the separator that says "here come the parameters"
- `q=joe` — a key-value pair: the parameter named `q` has the value `joe`

You can have multiple parameters separated by `&`:
```
https://example.com/contacts?q=joe&sort=name&page=2
```

**Two ways to create these in HTML:**

1. **Hardcoded in an anchor tag:**
```html
<a href="/contacts?q=joe">Search for Joe</a>
```

2. **Dynamically via a form with GET:**
```html
<form action="/contacts" method="get">
    <input type="text" name="q">
    <button>Search</button>
</form>
```

When you type "joe" and submit, the browser *automatically* builds the URL `/contacts?q=joe` from the form field's `name` attribute and its value.

Which part would you like to dig into more — the URL structure itself, or how forms build these automatically?

Url structure makes sense. Lets do the later (form)

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Let's trace through what happens step by step.

You have this form:
```html
<form action="/contacts" method="get">
    <input type="text" name="q">
    <button>Search</button>
</form>
```

When the user types "joe" into the input and clicks Search:

1. Browser collects all form fields with a `name` attribute
2. Builds key-value pairs: `fieldname=value`
3. Since `method="get"`, it appends these to the URL as a query string
4. Makes a GET request to `/contacts?q=joe`

If you had *two* inputs:
```html
<input type="text" name="q">
<input type="text" name="city">
```

And typed "joe" and "boston", the browser would build: `/contacts?q=joe&city=boston`

The `name` attribute becomes the key, the user's input becomes the value. The browser handles the `?`, the `=`, and the `&` automatically.

Does that mechanism make sense? Or would you like to see how it differs when `method="post"` is used instead?

Yes. Lets look at post request scenario

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠

With `method="post"`:

```html
<form action="/contacts" method="post">
    <input type="text" name="q">
    <input type="text" name="city">
    <button>Search</button>
</form>
```

When you type "joe" and "boston" and submit:

1. Browser still collects all fields with a `name` attribute
2. Still builds key-value pairs: `q=joe&city=boston`
3. But instead of appending to the URL, it puts this data in the **request body**
4. Makes a POST request to `/contacts` (URL stays clean, no query string)

Key differences:

| GET | POST |
|-----|------|
| Data in URL: `/contacts?q=joe&city=boston` | Data in request body, URL stays `/contacts` |
| Visible in browser address bar | Hidden from address bar |
| Bookmarkable | Not bookmarkable |
| Has length limits | No practical length limits |
| Good for searches, filters | Good for creating/modifying data |

Remember how we discussed that refreshing a POST can cause duplicate submissions? This is why — the browser would re-send that hidden body data.

Does the GET vs POST distinction feel clear now?

Yes it does. Thanks. 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You're welcome! Ready to continue with the Contact.app code, or is there anything else from this section you'd like to explore?

I think I get the idea. Can you highlight the salient points of this note? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Here are the key points:

**Same route, two behaviors** — The `/contacts` path handles both "show all contacts" AND "search results." It checks for a `q` query parameter and branches accordingly. This is a classic Web 1.0 pattern — reuse the same list display for both cases.

**Query strings in action** — Remember from our HTTP discussion? Here's a concrete example: `?q=joe` in the URL. The handler extracts this with `request.args.get("q")`. If present, search; if not, show all.

**Separation of concerns** — Notice how the handler doesn't know *how* contacts are stored (database, file, whatever). It just calls `Contact.search()` or `Contact.all()` and passes results to a template. The `Contact` class is the "model" that handles data; the handler just orchestrates.

**Templates render HTML** — The `render_template("index.html", contacts=contacts_set)` line is where hypermedia gets generated. The handler fetches data, then hands it to a template to produce HTML. This is the SSR pattern we discussed earlier.

**GET with query params** — This search uses a GET request (data in URL), not POST. That means search results are bookmarkable and shareable — you can send someone a link to `?q=joe`.

Ready to see what the template looks like, or any questions on this handler logic?

#### The list & search templates

Now that we have our handler logic written, we’ll create a template to render HTML in our response to the client. At a high level, our HTML response needs to have the following elements:

  * A list of any matching or all contacts.

  * A search box where a user may type and submit search terms.

  * A bit of surrounding “chrome”: a header and footer for the website that will be the same regardless of the page you are on.




We are using the Jinja2 templating language, which has the following features:

  * We can use double-curly braces, `{{ }}`, to embed expression values in the template.

  * we can use curly-percents, `{% %}`, for directives, like iteration or including other content.




Beyond this basic syntax, Jinja2 is very similar to other templating languages used to generate content, and should be easy to follow for most web developers.

Let’s look at the first few lines of code in the `index.html` template:
    
    
    {% extends 'layout.html' %} <1>
    
    {% block content %} <2>
    
      <form action="/contacts" method="get" class="tool-bar"> <3>
        <label for="search">Search Term</label>
        <input id="search" type="search" name="q"
          value="{{ request.args.get('q') or '' }}" /> <4>
        <input type="submit" value="Search"/>
      </form>

Start of index.html

  1. Set the layout template for this template.

  2. Delimit the content to be inserted into the layout.

  3. Create a search form that will issue an HTTP `GET` to `/contacts`.

  4. Create an input for a user to type search queries.




The first line of code references a base template, `layout.html`, with the `extends` directive. This layout template provides the layout for the page (again, sometimes called “the chrome”): it wraps the template content in an `<html>` tag, imports any necessary CSS and JavaScript in a `<head>` element, places a `<body>` tag around the main content and so forth. All the common content wrapped around the “normal” content for the entire application is located in this file.

The next line of code declares the `content` section of this template. This content block is used by the `layout.html` template to inject the content of `index.html` within its HTML.

Next we have our first bit of actual HTML, rather than just Jinja directives. We have a simple HTML form that allows you to search contacts by issuing a `GET` request to the `/contacts` path. The form itself contains a label and an input with the name “q.” This input’s value will be submitted with the `GET` request to the `/contacts` path, as a query string (since this is a `GET` request.)

Note that the value of this input is set to the Jinja expression `{{ request.args.get('q') or '' }}`. This expression is evaluated by Jinja and will insert the request value of “q” as the input’s value, if it exists. This will “preserve” the search value when a user does a search, so that when the results of a search are rendered the text input contains the term that was searched for. This makes for a better user experience since the user can see exactly what the current results match, rather than having a blank text box at the top of the screen.

Finally, we have a submit-type input. This will render as a button and, when it is clicked, it will trigger the form to issue an HTTP request.

This search interface forms the top of our contact page. Following it is a table of contacts, either all contacts or the contacts that match the search, if a search was done.

Here is what the template code for the contact table looks like:
    
    
    <table>
      <thead>
      <tr>
        <th>First <th>Last <th>Phone <th>Email <th/> <1>
      </tr>
      </thead>
      <tbody>
      {% for contact in contacts %} <2>
        <tr>
          <td>{{ contact.first }}</td>
          <td>{{ contact.last }}</td>
          <td>{{ contact.phone }}</td>
          <td>{{ contact.email }}</td> <3>
          <td><a href="/contacts/{{ contact.id }}/edit">Edit</a>
            <a href="/contacts/{{ contact.id }}">View</a></td> <4>
        </tr>
      {% endfor %}
      </tbody>
    </table>

The contacts table

  1. Output some headers for our table.

  2. Iterate over the contacts that were passed in to the template.

  3. Output the values of the current contact, first name, last name, etc.

  4. An “operations” column, with links to edit or view the contact details.




This is the core of the page: we construct a table with appropriate headers matching the data we are going to show for each contact. We iterate over the contacts that were passed into the template by the handler method using the `for` loop directive in Jinja2. We then construct a series of rows, one for each contact, where we render the first and last name, phone and email of the contact as table cells in the row.

Additionally, we have a table cell that includes two links:

  * A link to the “Edit” page for the contact, located at `/contacts/{{ contact.id }}/edit` (e.g., For the contact with id 42, the edit link will point to `/contacts/42/edit`)

  * A link to the “View” page for the contact `/contacts/{{ contact.id }}` (using our previous contact example, the view page would be at `/contacts/42`)




Finally, we have a bit of end-matter: a link to add a new contact and a Jinja2 directive to end the `content` block:
    
    
    <p>
        <a href="/contacts/new">Add Contact</a> <1>
      </p>
    
    {% endblock %} <2>

The “add contact” link

  1. Link to the page that allows you to create a new contact.

  2. The closing element of the `content` block.




And that’s our complete template. Using this simple server-side template, in combination with our handler method, we can respond with an HTML _representation_ of all the contacts requested. So far, so hypermedia.

[fig-contactapp] is what the template looks like, rendered with a bit of contact information.

![](https://hypermedia.systems/images/figure_2-2_table_etc.png#ai)

Contact.app

Now, our application won’t win any design awards at this point, but notice that our template, when rendered, provides all the functionality necessary to see all the contacts and search them, and also provides links to edit them, view details of them or even create a new one.

And it does all this without the client (that is, the browser) knowing a thing about what contacts are or how to work with them. Everything is encoded _in_ the hypermedia. A web browser accessing this application just knows how to issue HTTP requests and then render HTML, nothing more about the specifics of our applications end points or underlying domain model.

As simple as our application is at this point, it is thoroughly RESTful.



Can we explore this sentence for better clarity? `Note that the value of this input is set to the Jinja expression {{ request.args.get('q') or '' }}. This expression is evaluated by Jinja and will insert the request value of “q” as the input’s value, if it exists. This will “preserve” the search value when a user does a search, so that when the results of a search are rendered the text input contains the term that was searched for. This makes for a better user experience since the user can see exactly what the current results match, rather than having a blank text box at the top of the screen.`

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Sure! Let's trace through what happens with a concrete example.

**Scenario:** User searches for "joe"

1. User types "joe" in the search box and clicks Search
2. Browser makes GET request to `/contacts?q=joe`
3. Server runs the handler, finds matching contacts
4. Server renders the template to build HTML

When rendering, Jinja hits this line:
```html
<input id="search" type="search" name="q" 
       value="{{ request.args.get('q') or '' }}" />
```

Since `request.args.get('q')` is `"joe"`, Jinja produces:
```html
<input id="search" type="search" name="q" value="joe" />
```

**Result:** When the page loads with search results, the search box *still shows "joe"* — it's pre-filled.

**Without this trick:** The search box would be blank after every search. User sees results but has no reminder of what they searched for. Confusing!

**The `or ''` part:** If there's no `q` parameter (first visit to the page), `request.args.get('q')` returns `None`. The `or ''` ensures we get an empty string instead, so the input just has `value=""`.

Does that make the "preservation" idea clearer?

I get that. Seems like there are two versions of this `index.html` page. Once before the search and one after; is that right? How does the second part - after the search - not cause any issue when the template includes contact stuff? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great question! There's actually only *one* version of `index.html`. The trick is in what gets *passed* to it.

Look back at the handler:
```python
if search is not None:
    contacts_set = Contact.search(search)  # filtered results
else:
    contacts_set = Contact.all()           # all contacts

return render_template("index.html", contacts=contacts_set)
```

Either way, the template receives a variable called `contacts`. It could be:
- All 100 contacts (no search)
- Just 3 matching contacts (after searching "joe")
- Zero contacts (search with no matches)

The template doesn't care! It just loops over whatever it gets:
```html
{% for contact in contacts %}
    ...
{% endfor %}
```

If `contacts` has 3 items, you get 3 rows. If it has 0 items, the loop runs zero times — empty table body, no error.

Same template, different data. Does that clear it up?

What about before doing the search? We are stil in the same page right?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Right! Before any search, you visit `/contacts` (no query string).

In the handler:
- `request.args.get("q")` returns `None` (no `q` parameter exists)
- So `contacts_set = Contact.all()` runs
- Template gets all contacts

And in the template:
- `{{ request.args.get('q') or '' }}` evaluates to `''` (empty string)
- Search box is empty
- Table shows all contacts

Same page, same template, same handler — just different inputs leading to different outputs. First visit: no `q`, show all, empty search box. After search: has `q`, show filtered, search box pre-filled.

I think I see it. It's basically allowing the flexibility to just call /contacts or the ability to add additional search parameters and that could be done through the URL or via the the UI, the form to be precise. and if you were to do the UI way by default it just shows all the contacts and after that you can basically run the search with the form is that correct 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Exactly right! Three equivalent ways to get to the same filtered results:

1. **Direct URL:** Type `example.com/contacts?q=joe` in the address bar
2. **Share a link:** Click a link someone sent you with that URL
3. **Use the form:** Type "joe" in the search box, click Search

All three end up making the same GET request to `/contacts?q=joe`. The form is just a user-friendly way to *build* that URL.

And yes — visiting `/contacts` with no parameters gives you everything, then you can narrow down using the form. The form and the URL are just two interfaces to the same underlying mechanism.

We are using jinja2 templates to create html. A crud application to illustrate creating and sending html. Can you point out the important concepts in this note and anything worth dwelling

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Key concepts worth highlighting:

**Template inheritance** — `{% extends 'layout.html' %}` lets you define a base layout once (header, footer, CSS imports) and reuse it across all pages. The `{% block content %}` marks where page-specific content goes. DRY principle in action.

**The form is a hypermedia control** — Look at that search form: `<form action="/contacts" method="get">`. It tells the browser exactly where to go and how. The browser needs zero knowledge of "contacts" — it just follows the instructions embedded in the HTML.

**GET puts data in the URL** — The search input has `name="q"`, so when submitted, the browser creates `/contacts?q=whatever`. This is why search results are bookmarkable/shareable.

**Preserving user input** — `value="{{ request.args.get('q') or '' }}"` puts the search term back in the input box after results load. Small UX touch, but important.

**Dynamic links** — `/contacts/{{ contact.id }}/edit` generates different URLs for each contact. The hypermedia contains all the navigation paths — the client just follows links.

**The punchline at the end** — "The browser just knows how to issue HTTP requests and render HTML, nothing more." This is HATEOAS in practice. The server sends everything the client needs to know. No API documentation, no client-side routing logic, no domain knowledge baked into the browser.

Does the template inheritance (`extends`/`block`) pattern make sense, or should we trace through how the layout and content fit together?

### Adding A New Contact

The next bit of functionality that we will add to our application is the ability to add new contacts. To do this, we are going to need to handle that `/contacts/new` URL referenced in the “Add Contact” link above. Note that when a user clicks on that link, the browser will issue a `GET` request to the `/contacts/new` URL.

All the other routes we have so far use `GET` as well, but we are actually going to use two different HTTP methods for this bit of functionality: an HTTP `GET` to render a form for adding a new contact, and then an HTTP `POST` _to the same path_ to actually create the contact, so we are going to be explicit about the HTTP method we want to handle when we declare this route.

Here is the code:
    
    
    @app.route("/contacts/new", methods=['GET']) <1>
    def contacts_new_get():
        return render_template("new.html", contact=Contact()) <2>

The “new contact” GET route

  1. Declare a route, explicitly handling `GET` requests to this path.

  2. Render the `new.html` template, passing in a new contact object.




Simple enough. We just render a `new.html` template with a new Contact. (`Contact()` is how you construct a new instance of the `Contact` class in Python, if you aren’t familiar with it.)

While the handler code for this route is very simple, the `new.html` template is more complicated.

For the remaining templates we are going to omit the layout directive and the content block declaration, but you can assume they are the same unless we say otherwise. This will let us focus on the “meat” of the template.

If you are familiar with HTML you are probably expecting a form element here, and you will not be disappointed. We are going to use the standard form hypermedia control for collecting contact information and submitting it to the server.

Here is what our HTML looks like:
    
    
    <form action="/contacts/new" method="post"> <1>
      <fieldset>
        <legend>Contact Values</legend>
        <p>
          <label for="email">Email</label> <2>
          <input name="email" id="email"
            type="email" placeholder="Email"
            value="{{ contact.email or '' }}"> <3>
          <span class="error">
            {{ contact.errors['email'] }} <4>
          </span>
        </p>

The “new contact” form

  1. A form that submits to the `/contacts/new` path, using an HTTP `POST`.

  2. A label for the first form input.

  3. The first form input, of type email.

  4. Any error messages associated with this field.




In the first line of code we create a form that will submit back _to the same path_ that we are handling: `/contacts/new`. Rather than issuing an HTTP `GET` to this path, however, we will issue an HTTP `POST` to it. Using a `POST` in this manner will signal to the server that we want to create a new Contact, rather than get a form for creating one.

We then have a label (always a good practice!) and an input that captures the email of the contact being created. The name of the input is `email` and, when this form is submitted, the value of this input will be submitted in the `POST` request, associated with the `email` key.

Next we have inputs for the other fields for contacts:
    
    
    <p>
      <label for="first_name">First Name</label>
      <input name="first_name" id="first_name" type="text"
        placeholder="First Name" value="{{ contact.first or '' }}">
      <span class="error">{{ contact.errors['first'] }}</span>
    </p>
    <p>
      <label for="last_name">Last Name</label>
      <input name="last_name" id="last_name" type="text"
        placeholder="Last Name" value="{{ contact.last or '' }}">
      <span class="error">{{ contact.errors['last'] }}</span>
    </p>
    <p>
      <label for="phone">Phone</label>
      <input name="phone" id="phone" type="text" placeholder="Phone"
        value="{{ contact.phone or '' }}">
      <span class="error">{{ contact.errors['phone'] }}</span>
    </p>

Inputs and labels for the “new contact” form

Finally, we have a button that will submit the form, the end of the form tag, and a link back to the main contacts table:
    
    
    <button>Save</button>
      </fieldset>
    </form>
    
    <p>
      <a href="/contacts">Back</a>
    </p>

The submit button for the “new contact” form

It is easy to miss in this straight-forward example: we are seeing the flexibility of hypermedia in action.

If we add a new field, remove a field, or change the logic around how fields are validated or work with one another, this new state of affairs would be reflected in the new hypermedia representation given to users. A user would see the updated new form and be able to work with these new features, with no software update required.



I am unclear about why we need a `Contact` passed in the get request? I assume it would be empty so that the values are passed to create an Contact. What is the purpose of `contact.**` syntax in what is returned in the get request?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great question! You're right that on the *first* GET request, the Contact is empty — so `contact.email` is `None`, and `contact.email or ''` gives you an empty string in the input.

But notice those error spans: `{{ contact.errors['email'] }}`. Here's where it gets interesting.

**What happens if validation fails:**

1. User fills out form, clicks Save
2. POST to `/contacts/new`
3. Server tries to create contact, but email is invalid
4. Server re-renders `new.html` *with the same Contact object* — now it has:
   - The values the user entered (so they don't have to retype)
   - Error messages in `contact.errors`

So the template serves double duty:
- First visit (GET): empty Contact, empty fields, no errors
- After failed validation (POST): populated Contact, pre-filled fields, error messages shown

If you didn't pass a Contact on the initial GET, the template would break trying to access `contact.email`.

Does that explain why the empty Contact is needed upfront?

That makes sense. Did I miss that information in the last note? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Not quite — the note shows the *mechanism* (those `contact.errors['email']` spans) but doesn't explicitly explain the validation flow yet.

If you look at the heading after this section, it's "Handling the post to /contacts/new" — that's where they'll show what happens when validation fails and why the Contact object gets re-passed with errors populated.

You spotted the pattern from the code structure before they explained it!

We are looking at leveraging hypermedia sent by the server to do actions. No client side knowledge required besides rendering html and http knowledge. Is there anything else worth dwelling on here

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠

You've captured the core message. A few additional points worth pausing on:

**Same path, different methods** — `/contacts/new` handles both GET (show the form) and POST (process the submission). This is a common Web 1.0 pattern. The HTTP method distinguishes intent, not the URL.

**POST for creation** — Remember from our HTTP discussion: POST means "process this data" and is typically used for creating new resources. The form's `method="post"` triggers this. GET would put all the form data in the URL, which is wrong for creation (not bookmarkable, might have sensitive data, etc.).

**Error handling is built in** — Notice `{{ contact.errors['email'] }}` in each field. The template anticipates validation failures. If the POST fails, the server re-renders the same form with error messages *and* the user's entered values preserved. No JavaScript needed for this feedback loop.

**The flexibility point at the end** — This is subtle but important: if you add a field to the Contact model, you update the template, and *every user immediately gets the new form*. No app store update, no client-side code changes, no API version negotiation. The hypermedia *is* the interface contract.

Want to see how the POST handler processes the form submission, or is anything here unclear?

#### Handling the post to /contacts/new

The next step in our application is to handle the `POST` that this form makes to `/contacts/new`.

To do so, we need to add another route to our application that handles the `/contacts/new` path. The new route will handle an HTTP `POST` method instead of an HTTP `GET`. We will use the submitted form values to attempt to create a new Contact.

If we are successful in creating a Contact, we will redirect the user to the list of contacts and show a success message. If we aren’t successful, then we will render the new contact form again with whatever values the user entered and render error messages about what issues need to be fixed so that the user can correct them.

Here is our new request handler:
    
    
    @app.route("/contacts/new", methods=['POST'])
    def contacts_new():
        c = Contact(
          None,
          request.form['first_name'],
          request.form['last_name'],
          request.form['phone'],
          request.form['email']) <1>
        if c.save(): <2>
            flash("Created New Contact!")
            return redirect("/contacts") <3>
        else:
            return render_template("new.html", contact=c) <4>

The “new contact” controller code

  1. We construct a new contact object with the values from the form.

  2. We try to save it.

  3. On success, “flash” a success message & redirect to the `/contacts` page.

  4. On failure, re-render the form, showing any errors to the user.




The logic in this handler is a bit more complex than other methods we have seen. The first thing we do is create a new Contact, again using the `Contact()` syntax in Python to construct the object. We pass in the values that the user submitted in the form by using the `request.form` object, a feature provided by Flask.

This `request.form` allows us to access submitted form values in an easy and convenient way, by simply passing in the same name associated with the various inputs.

We also pass in `None` as the first value to the `Contact` constructor. This is the “id” parameter, and by passing in `None` we are signaling that it is a new contact, and needs to have an ID generated for it. (Again, we are not going into the details of how this model object is implemented, our only concern is using it to generate hypermedia responses.)

Next, we call the `save()` method on the Contact object. This method returns `true` if the save is successful, and `false` if the save is unsuccessful (for example, a bad email was submitted by the user).

If we are able to save the contact (that is, there were no validation errors), we create a _flash_ message indicating success, and redirect the browser back to the list page. A “flash” is a common feature in web frameworks that allows you to store a message that will be available on the _next_ request, typically in a cookie or in a session store.

Finally, if we are unable to save the contact, we re-render the `new.html` template with the contact. This will show the same template as above, but the inputs will be filled in with the submitted values, and any errors associated with the fields will be rendered to feedback to the user as to what validation failed.

**The Post/Redirect/Get Pattern**

This handler implements a common strategy in web 1.0-style development called the [Post/Redirect/Get](https://en.wikipedia.org/wiki/Post/Redirect/Get) or PRG pattern. By issuing an HTTP redirect once a contact has been created and forwarding the browser on to another location, we ensure that the `POST` does not end up in the browsers request cache.

This means that if the user accidentally (or intentionally) refreshes the page, the browser will not submit another `POST`, potentially creating another contact. Instead, it will issue the `GET` that we redirect to, which should be side-effect free.

We will use the PRG pattern in a few different places in this book.

OK, so we have our server-side logic set up to save contacts. And, believe it or not, this is about as complicated as our handler logic will get, even when we look at adding more sophisticated htmx-driven behaviors.



Can you dig a bit deeper into `If we are able to save the contact (that is, there were no validation errors), we create a flash message indicating success, and redirect the browser back to the list page. A “flash” is a common feature in web frameworks that allows you to store a message that will be available on the next request, typically in a cookie or in a session store.`. I understand the part about flashing a message but seems like there is more to it; especially the `store a message that will be available on the next request` which is unclear to me

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good catch — there's a subtle timing issue here that flash messages solve.

**The problem:**

1. User POSTs the form
2. Server saves contact successfully
3. Server redirects to `/contacts`
4. Browser makes a *new* GET request to `/contacts`
5. You want to show "Created New Contact!" on that page

But here's the issue: the redirect response itself can't contain the success message (it's just a "go over there" instruction). And by the time the GET request arrives, it's a *completely separate request* — the server has no memory of what just happened.

**The solution — flash:**

1. On the POST, server stores "Created New Contact!" somewhere temporary (cookie or session)
2. Server sends redirect
3. Browser makes GET to `/contacts`
4. Server checks the temporary storage, finds the message, displays it
5. Server *clears* that message so it doesn't show again on the next refresh

The message "survives" across the redirect because it's stored outside the request itself.

Does that explain why you need this mechanism rather than just returning the message directly?

I see. We want something that persists between two separate calls and flash seems to provide that

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Exactly! Flash is essentially a one-time note passed between requests. It persists just long enough to survive the redirect, then disappears after being displayed.

It's a neat solution to a constraint of HTTP: each request is stateless and independent, so you need *something* (cookie, session) to carry information across the gap.

Ready to move on, or anything else in this section?

Who is responsible for how the flash message appears to the user? Do frameworks allow customization - eg: different types of banner and such?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠

The *server* stores the flash message, but the *template* controls how it appears. The flash just holds the text (and sometimes a category like "success", "error", "warning").

In Flask, you'd typically have something in your layout template like:

```html
{% for message in get_flashed_messages() %}
    <div class="alert">{{ message }}</div>
{% endfor %}
```

For different styles, Flask lets you pass a category:

```python
flash("Created New Contact!", "success")
flash("Something went wrong", "error")
```

Then in the template you can style based on category — green banner for success, red for error, etc.

So frameworks give you the *mechanism* (store a message, retrieve it once), but *you* control the presentation via CSS and HTML in your templates. Want to see how that category-based styling might look?

I don't see this implemented in the template above. Is this for simplicity?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Right — they mentioned the template snippets omit the layout directive (`{% extends 'layout.html' %}`), and the flash message display would typically live in that `layout.html` file since it's common across all pages.

The notes focus on the "meat" of each individual template, skipping the shared chrome. The flash rendering code would be part of that shared layout they're not showing.

Good eye catching that gap!

This is a lengthly note. Can you please highlight the salient and important points worth dwelling on?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠

Key points worth dwelling on:

**Same path, different handler** — This is the second route for `/contacts/new`, but it handles POST instead of GET. Flask distinguishes them via `methods=['POST']`. Two handlers, one URL, separated by HTTP method.

**Form data access** — `request.form['first_name']` pulls submitted values by their input `name` attribute. The form's inputs become a dictionary on the server. Simple mapping.

**Branching on success/failure**:
- Success → `redirect("/contacts")` (send user elsewhere)
- Failure → `render_template("new.html", contact=c)` (re-show form with errors)

This branching is why the template had `{{ contact.errors['email'] }}` placeholders — they're used when validation fails.

**Post/Redirect/Get (PRG) pattern** — This is crucial! After a successful POST, you *redirect* rather than directly rendering a page. Why? If you render directly and the user refreshes, the browser re-submits the POST, potentially creating duplicate contacts. The redirect breaks this chain — refreshing just re-GETs the contacts list.

**Flash messages** — A way to pass a one-time message across the redirect. "Created New Contact!" is stored temporarily, displayed on the next page, then discarded.

The PRG pattern is something you'll see constantly in traditional web apps. Does that pattern make sense, or should we trace through what happens step-by-step when a user submits the form?

Can we trace through step by step the PRG pattern

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Sure! Let's trace through what happens when a user successfully creates a new contact:

**Step 1: User fills out the form and clicks "Save"**
- Browser has the form from GET `/contacts/new`
- User types in name, email, etc.
- Clicks the submit button

**Step 2: Browser sends POST to `/contacts/new`**
- HTTP request goes to server with form data in the body
- Server validates, creates the contact
- Success! Now what does the server return?

**Step 3: Server sends redirect response (not HTML!)**
- Instead of rendering a page, server returns HTTP status `302` or `303`
- Response includes header: `Location: /contacts`
- Body is essentially empty

**Step 4: Browser automatically follows the redirect**
- Browser sees the 3xx status code
- Makes a *new* GET request to `/contacts`
- This GET is now in the browser's history

**Step 5: Server returns the contacts list**
- Normal page render with the flash message "Created New Contact!"

**The key insight**: The browser's "current page" is now the result of a GET, not a POST. 

What happens if the user hits refresh at this point?

`Get` request is called instead of resubmitting?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Exactly! The browser refreshes the *last request it made*, which is now the GET to `/contacts`. No duplicate contact created.

**Without PRG** (if we just rendered the page directly after POST):
- Browser's "current page" = result of POST `/contacts/new`
- User refreshes → browser wants to re-POST
- Browser shows warning: "Resubmit form data?"
- User clicks OK → duplicate contact created

**With PRG**:
- Browser's "current page" = result of GET `/contacts`
- User refreshes → browser re-GETs `/contacts`
- Harmless, just fetches the list again

The redirect "launders" the browser history, replacing the POST with a safe GET. It's a small server-side trick that prevents a common user frustration.

Ready to move on to viewing and editing contacts?

How does the browser "know" to call the Get request when refreshed?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠

The browser doesn't really "know" anything special — it just remembers the last request it made.

When you navigate to a page, the browser stores that request in its history. The refresh button simply re-executes whatever request is currently in the address bar / history entry.

Here's the sequence:

1. POST to `/contacts/new` → server returns redirect (302/303)
2. Browser follows redirect, makes GET to `/contacts`
3. Browser's history entry now says: "I'm at `/contacts` via GET"
4. Refresh = re-run that GET

The redirect response (step 1→2) is transparent to the user — it happens so fast they don't notice. But it *replaces* what the browser considers the "current page."

The address bar tells the story: after the redirect, it shows `/contacts`, not `/contacts/new`. That's what the browser will refresh.

Does that click?

Ah! I think I see it. When refreshed, the last call which was a redirect gets called not the post which is second to last due to redirect? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Exactly right! The browser's history tracks the *final destination*, not the intermediate steps.

Think of it like:
- POST → redirect → GET (three steps, but browser only "remembers" the GET)

The POST and the redirect are like stepping stones — the browser landed on the GET and that's where it stays. Refresh re-runs the landing spot, not the journey.

You've got it. Ready to continue with viewing and editing contacts?

Before moving on, what if we had done post -> get (not redirect get)? The result would be the same right? Just not something that is done to make the implementation clean? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

The *initial* result would look the same to the user — they'd see the contacts list either way.

But the browser state is different:

**POST → render directly:**
- Address bar shows `/contacts/new`
- Browser remembers: "I'm here because of a POST"
- Refresh → "Do you want to resubmit form data?" warning
- User clicks OK → duplicate contact

**POST → redirect → GET:**
- Address bar shows `/contacts`
- Browser remembers: "I'm here because of a GET"
- Refresh → silently re-fetches the list
- No duplicate, no warning

So it's not just about clean implementation — it's about preventing a real user-facing bug. The PRG pattern exists specifically to avoid that "resubmit form data?" problem that confuses users and can corrupt data.

Makes sense. Thanks

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You're welcome! Ready to move on to viewing and editing contacts, or would you like to take a break here?

### Viewing The Details Of A Contact

The next piece of functionality we will implement is the detail page for a Contact. The user will navigate to this page by clicking the “View” link in one of the rows in the list of contacts. This will take them to the path `/contacts/<contact id>` (e.g., `/contacts/42`).

This is a common pattern in web development: contacts are treated as resources and the URLs around these resources are organized in a coherent manner.

  * If you wish to view all contacts, you issue a `GET` to `/contacts`.

  * If you want a hypermedia representation allowing you to create a new contact, you issue a `GET` to `/contacts/new`.

  * If you wish to view a specific contact (with, say, an id of `42), you issue a `GET` to `/contacts/42`.




**The Eternal Bike Shed of URL Design**

It is easy to quibble about the particulars of the path scheme you use for your application:

“Should we `POST` to `/contacts/new` or to `/contacts`?”

We have seen many arguments online and in person advocating for one approach versus another. We feel it is more important to understand the overarching idea of _resources_ and _hypermedia representations_ , rather than getting worked up about the smaller details of your URL design.

We recommend you just pick a reasonable, resource-oriented URL layout you like and then stay consistent. Remember, in a hypermedia system, you can always change your endpoints later, because you are using hypermedia as the engine of application state!

Our handler logic for the detail route is going to be _very_ simple: we just look the Contact up by id, which is embedded in the path of the URL for the route. To extract this ID we are going to need to introduce a final bit of Flask functionality: the ability to call out pieces of a path and have them automatically extracted and passed in to a handler function.

Here is what the code looks like, just a few lines of simple Python:
    
    
    @app.route("/contacts/<contact_id>") <1>
    def contacts_view(contact_id=0): <2>
        contact = Contact.find(contact_id) <3>
        return render_template("show.html", contact=contact) <4>

  1. Map the path, with a path variable named `contact_id`.

  2. The handler takes the value of this path parameter.

  3. Look up the corresponding contact.

  4. Render the `show.html` template.




You can see the syntax for extracting values from the path in the first line of code: you enclose the part of the path you wish to extract in `<>` and give it a name. This component of the path will be extracted and then passed into the handler function, via the parameter with the same name.

So, if you were to navigate to the path `/contacts/42`, the value `42` would be passed into the `contacts_view()` function for the value of `contact_id`.

Once we have the id of the contact we want to look up, we load it up using the `find` method on the `Contact` object. We then pass this contact into the `show.html` template and render a response.



What if the path was `/contacts/42/33` and we wanted the `33`; could we still do `@app.route("/contexts/something/<contact_id>` to get the desired value?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yes! You can place `<variable_name>` anywhere in the path, and you can have multiple of them.

For `/contacts/42/33`, you could do:

```python
@app.route("/contacts/<other_id>/<contact_id>")
def contacts_view(other_id, contact_id):
    # other_id = "42", contact_id = "33"
```

Or if you don't care about the `42`:

```python
@app.route("/contacts/<ignored>/<contact_id>")
def contacts_view(ignored, contact_id):
    # just use contact_id
```

You still need to capture it in the route pattern, even if you don't use it. Flask matches the whole path structure.

Can you highlight the salient bits of the last note. Anything worth dwelling on?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Key points:

**Path parameters** — The `<contact_id>` syntax extracts a value from the URL itself. So `/contacts/42` passes `42` to the handler as `contact_id`. This is different from query parameters (`?q=joe`) which come after a `?`.

**Resource-oriented URL pattern** — Notice the consistency:
- `/contacts` → all contacts (collection)
- `/contacts/new` → form to create one
- `/contacts/42` → a specific contact (single resource)

The URL structure reflects what you're looking at. Clean and predictable.

**The "bike shed" aside** — They're saying: don't obsess over URL design. People love arguing about whether to POST to `/contacts/new` or `/contacts`. Pick something sensible, stay consistent, and move on. Since hypermedia embeds the URLs anyway, clients don't hardcode them — you can always change later.

**The handler is dead simple** — Three lines: extract ID from path, look up contact, render template. No complexity. This is typical for "view" operations.

Ready to see what the detail template looks like?

### The Contact Detail Template

Our `show.html` template is relatively simple, just showing the same information as the table but in a slightly different format (perhaps for printing). If we add functionality like “notes” to the application later on, this will give us a good place to do so.

Again, we will omit the “chrome” of the template and focus on the meat:
    
    
    <h1>{{contact.first}} {{contact.last}}</h1>
    
    <div>
      <div>Phone: {{contact.phone}}</div>
      <div>Email: {{contact.email}}</div>
    </div>
    
    <p>
      <a href="/contacts/{{contact.id}}/edit">Edit</a>
      <a href="/contacts">Back</a>
    </p>

The “contact details” template

We simply render a First Name and Last Name header, with the additional contact information below it, and a couple of links: a link to edit the contact and a link to navigate back to the full list of contacts.



Can you highlight the salient bits of the last note. Anything worth dwelling on?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

This one is straightforward, but a couple of things to note:

**Just data display + navigation** — No forms here, just rendering values and providing links. This is a "read-only" view.

**Links as hypermedia controls** — Even this simple page has two actions embedded in the HTML:
- Edit this contact → `/contacts/{{contact.id}}/edit`
- Go back → `/contacts`

The user doesn't need to know these URLs exist — they're *in* the page. Click and go.

**Placeholder for future features** — They mention this is a good place to add things like "notes" later. Because it's hypermedia, you'd just update the template, add the data, and users immediately see it. No API versioning, no client updates.

Pretty minimal — ready to move on to editing and deleting contacts?

### Editing And Deleting A Contact

Next up we will tackle the functionality on the other end of that “Edit” link. Editing a contact is going to look very similar to creating a new contact. As with adding a new contact, we are going to need two routes that handle the same path, but using different HTTP methods: a `GET` to `/contacts/<contact_id>/edit` will return a form allowing you to edit the contact and a `POST` to that path will update it.

We are also going to piggyback the ability to delete a contact along with this editing functionality. To do this we will need to handle a `POST` to `/contacts/<contact_id>/delete`.

Let’s look at the code to handle the `GET`, which, again, will return an HTML representation of an editing interface for the given resource:
    
    
    @app.route("/contacts/<contact_id>/edit", methods=["GET"])
    def contacts_edit_get(contact_id=0):
        contact = Contact.find(contact_id)
        return render_template("edit.html", contact=contact)

The “edit contact” controller code

As you can see this looks a lot like our “Show Contact” functionality. In fact, it is nearly identical except for the template: here we render `edit.html` rather than `show.html`.

While our handler code looked similar to the “Show Contact” functionality, the `edit.html` template is going to look very similar to the template for the “New Contact” functionality: we will have a form that submits updated contact values to the same “edit” URL and that presents all the fields of a contact as inputs for editing, along with any error messages.

Here is the first bit of the form:
    
    
    <form action="/contacts/{{ contact.id }}/edit" method="post"> <1>
      <fieldset>
        <legend>Contact Values</legend>
        <p>
          <label for="email">Email</label>
          <input name="email" id="email" type="text"
            placeholder="Email" value="{{ contact.email }}"> <2>
          <span class="error">{{ contact.errors['email'] }}</span>
        </p>

The “edit contact” form start

  1. Issue a `POST` to the `/contacts/{{ contact.id }}/edit` path.

  2. As with the `new.html` page, the input is tied to the contact’s email.




This HTML is nearly identical to our `new.html` form, except that this form is going to submit a `POST` to a different path, based on the id of the contact that we want to update. (It’s worth mentioning here that, rather than `POST`, we would prefer to use a `PUT` or `PATCH`, but those are not available in plain HTML.)

Following this we have the remainder of our form, again very similar to the `new.html` template, and our button to submit the form.
    
    
    <p>
          <label for="first_name">First Name</label>
          <input name="first_name" id="first_name" type="text"
            placeholder="First Name" value="{{ contact.first }}">
          <span class="error">{{ contact.errors['first'] }}</span>
        </p>
        <p>
          <label for="last_name">Last Name</label>
          <input name="last_name" id="last_name" type="text"
            placeholder="Last Name" value="{{ contact.last }}">
          <span class="error">{{ contact.errors['last'] }}</span>
        </p>
        <p>
          <label for="phone">Phone</label>
          <input name="phone" id="phone" type="text"
            placeholder="Phone" value="{{ contact.phone }}">
          <span class="error">{{ contact.errors['phone'] }}</span>
        </p>
        <button>Save</button>
      </fieldset>
    </form>

The “edit contact” form body

In the final part of our template we have a small difference between the `new.html` and `edit.html`. Below the main editing form, we include a second form that allows you to delete a contact. It does this by issuing a `POST` to the `/contacts/<contact id>/delete` path. Just as we would prefer to use a `PUT` to update a contact, we would much rather use an HTTP `DELETE` request to delete one. Unfortunately that also isn’t possible in plain HTML.

To finish up the page, there is a simple hyperlink back to the list of contacts.
    
    
    <form action="/contacts/{{ contact.id }}/delete" method="post">
      <button>Delete Contact</button>
    </form>
    
    <p>
      <a href="/contacts/">Back</a>
    </p>

The “edit contact” form footer

Given all the similarities between the `new.html` and `edit.html` templates, you may be wondering why we are not _refactoring_ these two templates to share logic between them. That’s a good observation and, in a production system, we would probably do just that.

For our purposes, however, since our application is small and simple, we will leave the templates separate.

**Factoring Your Applications**

One thing that often trips people up who are coming to hypermedia applications from a JavaScript background is the notion of “components”. In JavaScript-oriented applications it is common to break your app up into small client-side components that are then composed together. These components are often developed and tested in isolation and provide a nice abstraction for developers to create testable code.

With Hypermedia-Driven Applications, in contrast, you factor your application on the server side. As we said, the above form could be refactored into a shared template between the edit and create templates, allowing you to achieve a reusable and DRY (Don’t Repeat Yourself) implementation.

Note that factoring on the server-side tends to be coarser-grained than on the client-side: you tend to split out common _sections_ rather than create lots of individual components. This has benefits (it tends to be simple) as well as drawbacks (it is not nearly as isolated as client-side components).

Overall, a properly factored server-side hypermedia application can be extremely DRY.



This was a hefty paragraph. Can you highlight the salient points and ideas worth dwelling on?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Key points:

**Same pattern as "new"** — GET returns a form, POST processes it. The edit handler is nearly identical to the view handler, just with a different template.

**HTML's method limitation** — They mention wanting to use PUT/PATCH for updates and DELETE for deletion, but plain HTML forms only support GET and POST. So they use POST for everything. This is a real limitation of HTML that htmx will later address.

**Two forms on one page** — The edit page has two separate forms:
1. Main form → POST to `/contacts/42/edit` (update)
2. Delete form → POST to `/contacts/42/delete` (delete)

Each form is its own hypermedia control with its own action. Simple and explicit.

**Server-side factoring vs client-side components** — This is worth dwelling on. In SPAs, you build tiny reusable components (button, input, card). In hypermedia apps, you factor at a coarser grain — shared *sections* or partial templates. Less isolation, but simpler. They acknowledge the `new.html` and `edit.html` templates are repetitive and could share code, but they're keeping it simple for teaching.

**DRY still applies** — Just happens on the server instead of the client.

Want to see how the POST handlers process the edit and delete actions?

#### Handling the post to /contacts/<contact_id>/edit

Next we need to handle the HTTP `POST` request that the form in our `edit.html` template submits. We will declare another route that handles the same path as the `GET` above.

Here is the new handler code:
    
    
    @app.route("/contacts/<contact_id>/edit", methods=["POST"]) <1>
    def contacts_edit_post(contact_id=0):
        c = Contact.find(contact_id) <2>
        c.update(
          request.form['first_name'],
          request.form['last_name'],
          request.form['phone'],
          request.form['email']) <3>
        if c.save(): <4>
            flash("Updated Contact!")
            return redirect("/contacts/" + str(contact_id)) <5>
        else:
            return render_template("edit.html", contact=c) <6>

  1. Handle a `POST` to `/contacts/<contact_id>/edit`.

  2. Look the contact up by id.

  3. Update the contact with the new information from the form.

  4. Attempt to save it.

  5. On success, flash a success message & redirect to the detail page.

  6. On failure, re-render the edit template, showing any errors.




The logic in this handler is very similar to the logic in the handler for adding a new contact. The only real difference is that, rather than creating a new Contact, we look the contact up by id and then call the `update()` method on it with the values that were entered in the form.

Once again, this consistency between our CRUD operations is one of the nice and simplifying aspects of traditional CRUD web applications.



Can you highlight the salient points and ideas worth dwelling on?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Key points:

**PRG pattern again** — On success, they redirect to `/contacts/42` (the detail page), not back to the list. Flash message, redirect, done. Same pattern as creating a new contact.

**Find, then update** — Unlike "new" where you create a contact from scratch, here you:
1. Look up the existing contact by ID
2. Update its values
3. Try to save

The contact already exists; you're modifying it.

**Failure handling is identical** — If validation fails, re-render `edit.html` with the contact (which now has error messages attached). User sees their input preserved plus what went wrong.

**CRUD consistency** — They point out how similar this is to the "new" handler. Create and Update follow nearly the same flow: get data from form, try to save, redirect on success, re-render with errors on failure. This predictability is a feature of traditional web apps.

Ready to see the delete handler?

### Deleting A Contact

We piggybacked contact delete functionality into the same template used to edit a contact. This second form will issue an HTTP `POST` to `/contacts/<contact_id>/delete`, and we will need to create a handler for that path as well.

Here is what the controller looks like:
    
    
    @app.route("/contacts/<contact_id>/delete", methods=["POST"]) <3>
    def contacts_delete(contact_id=0):
        contact = Contact.find(contact_id)
        contact.delete() <2>
        flash("Deleted Contact!")
        return redirect("/contacts") <3>

The “delete contact” controller code

  1. Handle a `POST` the `/contacts/<contact_id>/delete` path.

  2. Look up and then invoke the `delete()` method on the contact.

  3. Flash a success message and redirect to the main list of contacts.




The handler code is very simple since we don’t need to do any validation or conditional logic: we simply look up the contact the same way we have been doing in our other handlers and invoke the `delete()` method on it, then redirect back to the list of contacts with a success flash message.

No need for a template in this case, the contact is gone.



Can you highlight the salient points and ideas worth dwelling on? Also we are not testing for the case where the contact doesn't exist. Is this because we are in a contacts view page which effectively doesn't allow calling delete on a non-contact

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠

Key points:

**Simplest handler so far** — No validation, no branching. Find contact, delete it, redirect. Done.

**PRG pattern yet again** — Even for delete, they redirect to `/contacts` rather than rendering directly. Prevents "resubmit" issues if user refreshes.

**No template needed** — The resource is gone, so there's nothing to show. Just redirect elsewhere.

---

On your question about missing contacts: You're exactly right! This is "happy path" code for teaching purposes. In the hypermedia approach, the delete button only appears on pages for contacts that *exist* — you navigated there via a link that the server generated. So the assumption is: if you're on this page, the contact is real.

But in production, you'd still want to handle edge cases:
- User bookmarked the edit page, contact was deleted by someone else
- User has two tabs open, deletes in one, then tries to delete in the other
- Malicious user crafts a URL manually

You'd typically add a check: if `Contact.find()` returns nothing, show a 404 or flash an error and redirect.

Good catch — you're thinking like someone who has to maintain real software!

### Contact.app…​ Implemented!

And, well…​ believe it or not, that’s our entire contact application!

If you’ve struggled with parts of the code so far, don’t worry: we don’t expect you to be a Python or Flask expert (we aren’t!). You just need a basic understanding of how they work to benefit from the remainder of the book.

This is a small and simple application, but it does demonstrate many of the aspects of traditional, web 1.0 applications: CRUD, the Post/Redirect/Get pattern, working with domain logic in a controller, organizing our URLs in a coherent, resource-oriented manner.

And, furthermore, this is a deeply _Hypermedia-Driven_ web application. Without thinking about it very much, we have been using REST, HATEOAS and all the other hypermedia concepts we discussed earlier. We would bet that this simple little contact app of ours is more RESTful than 99% of all JSON APIs ever built!

Just by virtue of using a _hypermedia_ , HTML, we naturally fall into the RESTful network architecture.

So that’s great. But what’s the matter with this little web app? Why not end here and go off to develop web 1.0 style applications?

Well, at some level, nothing is wrong with it. Particularly for an application as simple as this one, the older way of building web apps might be a perfectly acceptable approach.

However, our application does suffer from that “clunkiness” that we mentioned earlier when discussing web 1.0 applications: every request replaces the entire screen, introducing a noticeable flicker when navigating between pages. You lose your scroll state. You have to click around a bit more than you might in a more sophisticated web application.

Contact.app, at this point, just doesn’t feel like a “modern” web application.

Is it time to reach for a JavaScript framework and JSON APIs to make our contact application more interactive?

No. No it isn’t.

It turns out that we can improve the user experience of this application while retaining its fundamental hypermedia architecture.

In the next few chapters we will look at [htmx](https://htmx.org), a hypermedia-oriented library that will let us improve our contact application while retaining the hypermedia-based approach we have used so far.



Can you highlight the salient points in the last note.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠

Key points:

**It's a complete app** — Despite being simple, Contact.app covers all of CRUD with proper patterns (PRG, resource-oriented URLs, error handling). It's production-worthy in structure, just minimal in features.

**Accidentally RESTful** — By using HTML (a hypermedia), they naturally ended up with a truly RESTful app without trying. HATEOAS, self-describing responses, loose coupling — all just happened. They claim it's more RESTful than 99% of JSON APIs, and they're probably right.

**But it's "clunky"** — Full page reloads on every action, flicker between pages, scroll position lost. It *works*, but doesn't feel modern.

**The punchline** — Is it time to reach for React and JSON APIs? *No.* They're going to improve the UX while *keeping* the hypermedia architecture. That's what htmx is for.

This is the transition point: we've seen what plain HTML can do and where it falls short. Next comes htmx to fill those gaps without abandoning the hypermedia approach.

Ready to move into htmx territory?

I mostly skimmed the code of this chapter. Is it necessary to go back and understand every line in order to grasp the salient and important points of this chapter?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

No, you don't need to understand every line. The code is just a vehicle for the concepts.

What matters from this chapter:

1. **Routes map URLs to handlers** — path comes in, function runs, response goes out
2. **GET for reading, POST for changing** — same URL can have both
3. **Templates generate HTML** — server builds the hypermedia, client just renders it
4. **PRG pattern** — POST, then redirect to GET, prevents duplicate submissions
5. **Forms are hypermedia controls** — they encode where to go and how, right in the HTML
6. **Everything is RESTful by default** — just by using HTML properly

If you can explain those six points, you've got the chapter. The Flask/Jinja syntax is incidental — you could implement the same patterns in any language.

Does that feel like a sufficient grasp, or is there a specific part you'd like to revisit?

Can you create a summary of this page for a future language model. The summary is meant to encapsulate everything up to now acting as a strong jumping off point for the next chapter. You can use what you see in the beginning of this dialog for inspiration.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Here's a summary to add to the existing one:

---

**Chapter Covered**: "A Web 1.0 Application" from "Hypermedia Systems"

**What Was Built**: Contact.app — a complete CRUD application for managing contacts using Flask (Python web framework) and Jinja2 templates.

**Key Concepts & Patterns**:

1. **Routes & Handlers**: URLs map to Python functions via decorators (`@app.route("/contacts")`). The handler processes the request and returns a response (HTML or redirect).

2. **Same Path, Different Methods**: A single URL can have multiple handlers distinguished by HTTP method. `/contacts/new` uses GET to show a form, POST to process submission.

3. **Server-Side Rendering (SSR)**: Templates generate complete HTML on the server. The browser receives ready-to-render hypermedia, not JSON + client code.

4. **Template Inheritance**: Base layouts (`layout.html`) provide common "chrome"; child templates inject content into `{% block %}` sections. DRY on the server.

5. **Post/Redirect/Get (PRG)**: After a successful POST, redirect to a GET. This makes the "current page" a safe GET request, so refresh doesn't resubmit the form. Prevents duplicate data creation.

6. **Query Parameters vs Path Parameters**: 
   - Query: `?q=joe` for search/filtering
   - Path: `/contacts/42` for identifying specific resources

7. **Forms as Hypermedia Controls**: `<form action="/contacts/new" method="post">` encodes everything the browser needs — destination, method, input names. No client-side logic required.

8. **Error Handling Pattern**: If validation fails, re-render the same form with the contact object (which now contains errors). User sees their input preserved + error messages.

9. **Resource-Oriented URLs**:
   - `/contacts` — collection
   - `/contacts/new` — form to create
   - `/contacts/42` — single resource
   - `/contacts/42/edit` — form to edit
   - `/contacts/42/delete` — delete action

10. **HTML's Limitations**: Forms only support GET and POST. No PUT, PATCH, or DELETE. This is a real constraint that htmx will address.

11. **Accidentally RESTful**: By using hypermedia (HTML), the app naturally satisfies REST constraints — HATEOAS, self-descriptive messages, loose coupling — without explicit effort.

12. **Server-Side Factoring**: Unlike SPAs with tiny client-side components, hypermedia apps factor at coarser grain — shared template sections rather than isolated widgets.

13. **The Trade-off**: The app is architecturally sound and truly RESTful, but "clunky" — full page reloads, flicker, lost scroll state. It works but doesn't feel modern.

**Where We're Headed**: The next chapters introduce htmx, which extends HTML's capabilities to make the app feel modern while preserving the hypermedia architecture.

---

Want me to adjust the level of detail or emphasis on anything?

This is fine. Can you combine this summary with the summary up to now in the beginning of the dialog. 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Here's the combined summary:

---

**Learner Context**: New to web programming, comfortable with Python, prefers learning through discussion and exploring concepts.

**Chapters Covered**: "Hypermedia: A Reintroduction", "Components of a Hypermedia System", and "A Web 1.0 Application" from "Hypermedia Systems"

**Key Concepts Discussed**:

1. **Hypermedia Definition**: Media with non-linear branching (e.g., hyperlinks). A **hypermedia control** encodes interaction information directly and completely within itself — self-describing, no external knowledge needed. Think about the *whole system*: HTML + HTTP + browser + server working together.

2. **HTML's Two Hypermedia Controls**: Anchor tags (navigation, GET) and form tags (data submission, GET/POST). Forms enabled the web to become an *application* platform, not just documents. GET appends data to URL (bookmarkable); POST sends data in body. POST-Redirect-GET pattern prevents duplicate submissions.

3. **SPAs vs Hypermedia**: SPAs exchange JSON data, require client-side code with intimate knowledge of API structure (**tight coupling**). Hypermedia exchanges HTML with embedded controls — server can change without breaking client (**loose coupling**). SPAs shift code/complexity to the browser.

4. **Why SPAs Won**: HTML froze as hypermedia in the mid-90s; user experience demands grew; JavaScript filled the gap. Won on *user experience*, not architectural superiority.

5. **Hypermedia-Driven Applications (HDAs)**: Apps using hypermedia exchanges as primary server communication. Libraries like **htmx** extend HTML's capabilities with declarative attributes (`hx-get`, `hx-target`). **Important**: browsers don't natively understand these attributes — the htmx library scans for them, attaches behavior, handles requests/swaps; the browser just renders the resulting HTML.

6. **Transitional vs Hypermedia-First**: Rich Harris's "transitional" approach = SPA as home base with MPA sprinkled in. The book's approach = **hypermedia as default**, SPA only when genuinely needed. The mindset is flipped.

7. **When to Use What**: Plain HTML → many apps. HTML + htmx → most interactivity needs. SPA → only when required (e.g., spreadsheets with cascading updates on every keystroke). **Can mix**: hypermedia for simple parts, SPA for complex features — save your complexity budget.

8. **Browser Benefits "For Free"**: Back button, deep linking, caching — all work naturally with hypermedia; must be manually reimplemented in SPAs.

9. **Four Components of a Hypermedia System**: Hypermedia (HTML), network protocol (HTTP), hypermedia server (responds to requests), hypermedia client (browser interprets responses). All four must work together.

10. **URLs**: Uniform Resource Locators uniquely identify resources. Components: scheme (protocol), host (domain), port, path, query, fragment. Relative URLs inherit from current document.

11. **HTTP Basics**: Simple request/response protocol. Request headers (like `Accept`) tell server preferences; response headers provide metadata. Content negotiation allows same URL to return different representations.

12. **HTTP Methods**: GET (read), POST (create/catch-all), PUT (replace), PATCH (partial update), DELETE (remove). Map roughly to CRUD. HTML only supports GET and POST natively — a limitation htmx addresses. Methods are conventions, not enforced constraints.

13. **HTTP Response Codes**: 1xx informational, 2xx success, 3xx redirect, 4xx client error, 5xx server error. Subtle differences matter (302 vs 303). Using codes properly goes "with the grain" of the web.

14. **Caching**: `Cache-Control` header indicates cacheability. `Vary` header tells caches which request headers affect the response (important for htmx's `HX-Request` header).

15. **Hypermedia Servers**: Any language works — hypermedia frees you from JavaScript lock-in. "HOWL" = Hypermedia On Whatever you'd Like.

16. **Hypermedia Clients**: Must generically interpret hypermedia controls. Browsers don't know what a "contact" is — they just render HTML. JSON API clients expect fixed formats, which is why HATEOAS hasn't caught on there.

17. **REST (Original Meaning)**: Fielding's dissertation described the pre-API web. Most "REST APIs" today aren't RESTful by his definition.

18. **REST Constraints**:
    - *Client-Server*: Obvious for web apps
    - *Statelessness*: Each request self-contained. Sessions (cookies) technically violate this — pragmatic trade-off
    - *Caching*: Responses indicate cacheability
    - *Uniform Interface*: The core innovation (see below)
    - *Layered System*: Intermediaries (CDNs, proxies) allowed
    - *Code-On-Demand*: Scripting is optional — should augment, not replace hypermedia

19. **Uniform Interface Sub-constraints**:
    - *Identification of resources*: URLs uniquely identify resources
    - *Manipulation through representations*: You exchange representations, not resources directly
    - *Self-descriptive messages*: All info needed is in the response itself
    - *HATEOAS*: Hypermedia As The Engine of Application State — available actions encoded in HTML, not in client code

20. **HATEOAS & API Churn**: Hypermedia APIs don't need versioning. Server changes, client automatically gets new controls. JSON APIs require documentation exchange and coordinated updates.

21. **Core Trade-off**: Hypermedia is less efficient (more bytes) but vastly more flexible — clients need zero domain knowledge, just render what arrives.

---

**Chapter 3: A Web 1.0 Application — Contact.app**

22. **Routes & Handlers**: URLs map to Python functions via decorators (`@app.route("/contacts")`). The handler processes the request and returns a response (HTML or redirect).

23. **Same Path, Different Methods**: A single URL can have multiple handlers distinguished by HTTP method. `/contacts/new` uses GET to show a form, POST to process submission.

24. **Server-Side Rendering (SSR)**: Templates generate complete HTML on the server. The browser receives ready-to-render hypermedia, not JSON + client code.

25. **Template Inheritance**: Base layouts (`layout.html`) provide common "chrome"; child templates inject content into `{% block %}` sections. DRY on the server.

26. **Post/Redirect/Get (PRG)**: After a successful POST, redirect to a GET. This makes the "current page" a safe GET request, so refresh doesn't resubmit the form. The browser's history tracks the *final destination* (the GET), not the POST — so refresh re-runs the harmless GET.

27. **Query Parameters vs Path Parameters**: 
    - Query: `?q=joe` for search/filtering (after the `?`)
    - Path: `/contacts/42` for identifying specific resources (in the URL structure)

28. **Forms as Hypermedia Controls**: `<form action="/contacts/new" method="post">` encodes everything the browser needs — destination, method, input names. No client-side logic required.

29. **Error Handling Pattern**: If validation fails, re-render the same form with the contact object (which now contains errors). User sees their input preserved + error messages. No JavaScript needed.

30. **Resource-Oriented URLs**:
    - `/contacts` — collection
    - `/contacts/new` — form to create
    - `/contacts/42` — single resource
    - `/contacts/42/edit` — form to edit
    - `/contacts/42/delete` — delete action

31. **HTML's Method Limitations**: Forms only support GET and POST. No PUT, PATCH, or DELETE natively. This is a real constraint that htmx will address.

32. **Accidentally RESTful**: By using hypermedia (HTML), the app naturally satisfies REST constraints — HATEOAS, self-descriptive messages, loose coupling — without explicit effort. More RESTful than most JSON APIs.

33. **Server-Side Factoring**: Unlike SPAs with tiny client-side components, hypermedia apps factor at coarser grain — shared template sections rather than isolated widgets. Simpler but less isolated.

34. **The Web 1.0 Trade-off**: The app is architecturally sound and truly RESTful, but "clunky" — full page reloads, flicker, lost scroll state. It works but doesn't feel modern.

**Where We're Headed**: The next chapters introduce htmx, which extends HTML's capabilities to make the app feel modern while preserving the hypermedia architecture.

---

Does this capture everything appropriately?

Yes. Since we do not mention chapter number for previous summary lets remove that for chapter 3. Please create a note with the summary once you have made that fix. 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠



<details class='tool-usage-details'>

```json
{
  "id": "toolu_011nin1wN9gSAugA8Lgjeuv5",
  "call": {
    "function": "add_msg",
    "arguments": {
      "content": "Overall summary upto this point.\n---\n\n**Learner Context**: New to web programming, comfortable with Python, prefers learning through discussion and exploring concepts.\n\n**Chapters Covered**: \"Hypermedia: A Reintroduction\", \"Components of a Hypermedia System\", and \"A Web 1.0 Application\" from \"Hypermedia Systems\"\n\n**Key Concepts Discussed**:\n\n1. **Hypermedia Definition**: Media with non-linear branching (e.g., hyperlinks). A **hypermedia control** encodes interaction information directly and completely within itself \u2014 self-describing, no external knowledge needed. Think about the *whole system*<TRUNCATED>",
      "msg_type": "note"
    }
  },
  "result": "'_d1d062d4'"
}
```

</details>

Done! I've created a note with the combined summary, removing the chapter number header for consistency.

Overall summary upto this point.
---

**Learner Context**: New to web programming, comfortable with Python, prefers learning through discussion and exploring concepts.

**Chapters Covered**: "Hypermedia: A Reintroduction", "Components of a Hypermedia System", and "A Web 1.0 Application" from "Hypermedia Systems"

**Key Concepts Discussed**:

1. **Hypermedia Definition**: Media with non-linear branching (e.g., hyperlinks). A **hypermedia control** encodes interaction information directly and completely within itself — self-describing, no external knowledge needed. Think about the *whole system*: HTML + HTTP + browser + server working together.

2. **HTML's Two Hypermedia Controls**: Anchor tags (navigation, GET) and form tags (data submission, GET/POST). Forms enabled the web to become an *application* platform, not just documents. GET appends data to URL (bookmarkable); POST sends data in body. POST-Redirect-GET pattern prevents duplicate submissions.

3. **SPAs vs Hypermedia**: SPAs exchange JSON data, require client-side code with intimate knowledge of API structure (**tight coupling**). Hypermedia exchanges HTML with embedded controls — server can change without breaking client (**loose coupling**). SPAs shift code/complexity to the browser.

4. **Why SPAs Won**: HTML froze as hypermedia in the mid-90s; user experience demands grew; JavaScript filled the gap. Won on *user experience*, not architectural superiority.

5. **Hypermedia-Driven Applications (HDAs)**: Apps using hypermedia exchanges as primary server communication. Libraries like **htmx** extend HTML's capabilities with declarative attributes (`hx-get`, `hx-target`). **Important**: browsers don't natively understand these attributes — the htmx library scans for them, attaches behavior, handles requests/swaps; the browser just renders the resulting HTML.

6. **Transitional vs Hypermedia-First**: Rich Harris's "transitional" approach = SPA as home base with MPA sprinkled in. The book's approach = **hypermedia as default**, SPA only when genuinely needed. The mindset is flipped.

7. **When to Use What**: Plain HTML → many apps. HTML + htmx → most interactivity needs. SPA → only when required (e.g., spreadsheets with cascading updates on every keystroke). **Can mix**: hypermedia for simple parts, SPA for complex features — save your complexity budget.

8. **Browser Benefits "For Free"**: Back button, deep linking, caching — all work naturally with hypermedia; must be manually reimplemented in SPAs.

9. **Four Components of a Hypermedia System**: Hypermedia (HTML), network protocol (HTTP), hypermedia server (responds to requests), hypermedia client (browser interprets responses). All four must work together.

10. **URLs**: Uniform Resource Locators uniquely identify resources. Components: scheme (protocol), host (domain), port, path, query, fragment. Relative URLs inherit from current document.

11. **HTTP Basics**: Simple request/response protocol. Request headers (like `Accept`) tell server preferences; response headers provide metadata. Content negotiation allows same URL to return different representations.

12. **HTTP Methods**: GET (read), POST (create/catch-all), PUT (replace), PATCH (partial update), DELETE (remove). Map roughly to CRUD. HTML only supports GET and POST natively — a limitation htmx addresses. Methods are conventions, not enforced constraints.

13. **HTTP Response Codes**: 1xx informational, 2xx success, 3xx redirect, 4xx client error, 5xx server error. Subtle differences matter (302 vs 303). Using codes properly goes "with the grain" of the web.

14. **Caching**: `Cache-Control` header indicates cacheability. `Vary` header tells caches which request headers affect the response (important for htmx's `HX-Request` header).

15. **Hypermedia Servers**: Any language works — hypermedia frees you from JavaScript lock-in. "HOWL" = Hypermedia On Whatever you'd Like.

16. **Hypermedia Clients**: Must generically interpret hypermedia controls. Browsers don't know what a "contact" is — they just render HTML. JSON API clients expect fixed formats, which is why HATEOAS hasn't caught on there.

17. **REST (Original Meaning)**: Fielding's dissertation described the pre-API web. Most "REST APIs" today aren't RESTful by his definition.

18. **REST Constraints**:
    - *Client-Server*: Obvious for web apps
    - *Statelessness*: Each request self-contained. Sessions (cookies) technically violate this — pragmatic trade-off
    - *Caching*: Responses indicate cacheability
    - *Uniform Interface*: The core innovation (see below)
    - *Layered System*: Intermediaries (CDNs, proxies) allowed
    - *Code-On-Demand*: Scripting is optional — should augment, not replace hypermedia

19. **Uniform Interface Sub-constraints**:
    - *Identification of resources*: URLs uniquely identify resources
    - *Manipulation through representations*: You exchange representations, not resources directly
    - *Self-descriptive messages*: All info needed is in the response itself
    - *HATEOAS*: Hypermedia As The Engine of Application State — available actions encoded in HTML, not in client code

20. **HATEOAS & API Churn**: Hypermedia APIs don't need versioning. Server changes, client automatically gets new controls. JSON APIs require documentation exchange and coordinated updates.

21. **Core Trade-off**: Hypermedia is less efficient (more bytes) but vastly more flexible — clients need zero domain knowledge, just render what arrives.

22. **Routes & Handlers**: URLs map to Python functions via decorators (`@app.route("/contacts")`). The handler processes the request and returns a response (HTML or redirect).

23. **Same Path, Different Methods**: A single URL can have multiple handlers distinguished by HTTP method. `/contacts/new` uses GET to show a form, POST to process submission.

24. **Server-Side Rendering (SSR)**: Templates generate complete HTML on the server. The browser receives ready-to-render hypermedia, not JSON + client code.

25. **Template Inheritance**: Base layouts (`layout.html`) provide common "chrome"; child templates inject content into `{% block %}` sections. DRY on the server.

26. **Post/Redirect/Get (PRG)**: After a successful POST, redirect to a GET. This makes the "current page" a safe GET request, so refresh doesn't resubmit the form. The browser's history tracks the *final destination* (the GET), not the POST — so refresh re-runs the harmless GET.

27. **Query Parameters vs Path Parameters**: 
    - Query: `?q=joe` for search/filtering (after the `?`)
    - Path: `/contacts/42` for identifying specific resources (in the URL structure)

28. **Forms as Hypermedia Controls**: `<form action="/contacts/new" method="post">` encodes everything the browser needs — destination, method, input names. No client-side logic required.

29. **Error Handling Pattern**: If validation fails, re-render the same form with the contact object (which now contains errors). User sees their input preserved + error messages. No JavaScript needed.

30. **Resource-Oriented URLs**:
    - `/contacts` — collection
    - `/contacts/new` — form to create
    - `/contacts/42` — single resource
    - `/contacts/42/edit` — form to edit
    - `/contacts/42/delete` — delete action

31. **HTML's Method Limitations**: Forms only support GET and POST. No PUT, PATCH, or DELETE natively. This is a real constraint that htmx will address.

32. **Accidentally RESTful**: By using hypermedia (HTML), the app naturally satisfies REST constraints — HATEOAS, self-descriptive messages, loose coupling — without explicit effort. More RESTful than most JSON APIs.

33. **Server-Side Factoring**: Unlike SPAs with tiny client-side components, hypermedia apps factor at coarser grain — shared template sections rather than isolated widgets. Simpler but less isolated.

34. **The Web 1.0 Trade-off**: The app is architecturally sound and truly RESTful, but "clunky" — full page reloads, flicker, lost scroll state. It works but doesn't feel modern.

**Where We're Headed**: The next chapters introduce htmx, which extends HTML's capabilities to make the app feel modern while preserving the hypermedia architecture.

## HTML Notes: Framework Soup

Components encapsulate a section of a page along with its dynamic behavior. While encapsulating behavior is a good way to organize code, it can also separate elements from their surrounding context, which can lead to wrong or inadequate relationships between elements. The result is what one might call _component soup_ , where information is hidden in component state, rather than being present in the HTML, which is now incomprehensible due to missing context.

Before you reach for components for reuse, consider your options. Lower-level mechanisms often (allow you to) produce better HTML. In some cases, components can actually _improve_ the clarity of your HTML.

> The fact that the HTML document is something that you barely touch, because everything you need in there will be injected via JavaScript, puts the document and the page structure out of focus.

 Manuel Matuzović, [Why I’m not the biggest fan of Single Page Applications](https://www.matuzo.at/blog/2023/single-page-applications-criticism)

In order to avoid `<div>` soup (or Markdown soup, or Component soup), you need to be aware of the markup you’re producing and be able to change it.

Some SPA frameworks, and some web components, make this more difficult by putting layers of abstraction between the code the developer writes and the generated markup.

While these abstractions can allow developers to create richer UI or work faster, their pervasiveness means that developers can lose sight of the actual HTML (and JavaScript) being sent to clients. Without diligent testing, this leads to inaccessibility, poor SEO, and bloat.

[Previous: Components Of A Hypermedia System](/components-of-a-hypermedia-system/)

[Next: Hypermedia-Driven Web Applications With Htmx](/part/htmx/)

[Hypermedia Systems](/) [Contents](/book/contents/)


What is "markup" as it is used in the last note? What is markdown and how are they different? How do they all relate to html?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great question — these terms are related but distinct.

**Markup**: A general term for any system of annotations that describe structure or formatting within text. The "marks" tell a processor how to display or interpret the content. HTML, XML, LaTeX — all are markup languages.

**HTML (HyperText Markup Language)**: A specific markup language for web pages. Uses tags like `<p>`, `<h1>`, `<a>` to define structure. Browsers know how to render it.

**Markdown**: A lightweight markup language designed to be *human-readable* even in its raw form. Uses simple symbols like `#` for headings, `*` for emphasis, `-` for lists. It's typically *converted to HTML* for display.

```
Markdown:     # Hello        →    HTML: <h1>Hello</h1>
Markdown:     **bold**       →    HTML: <strong>bold</strong>
```

So the relationship is:
- Markup = the broad category
- HTML = a specific markup language (what browsers render)
- Markdown = a simpler markup language that gets converted to HTML

The note's warning about "Markdown soup" or "Component soup" is saying: whatever tool you use to *generate* HTML, don't lose sight of the actual HTML being produced — that's what matters for accessibility and semantics.

Does the hierarchy make sense?

It does. Thanks